In [1]:
%pip install pandas_datareader pykrx yfinance pyarrow ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 14.6 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=8e78caf45aa5d69cf70ca20dfae167865394253b613db6e5f31db97b181fc06e
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


# src

## common

### dtype.py

In [2]:
class dDict(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for key, value in kwargs.items():
            if isinstance(value, dict):
                value = dDict(**value)
            self[key] = value

    def __iter__(self):
        return iter(self.items())

    def __getattr__(self, attr):
        try:
            return self[attr]
        except KeyError:
            raise AttributeError(f"No such attribute: {attr}")

    def __setattr__(self, attr, value):
        self[attr] = value


### env.py

In [3]:
# try:
#     from .dtype import dDict
# except ImportError:
#     from src.common.dtype import dDict
from datetime import datetime, timezone, timedelta
import os


# DATETIME CLOCK: %Y-%m-%d %H:%M:%s+09:00
CLOCK = lambda: datetime.now(timezone(timedelta(hours=9)))

# ENVIRONMENT VARIABLE: DETECTING CURRENT RUNNER
ENV   = "local"
if any([key.lower().startswith('colab') for key in os.environ]):
    ENV = 'google_colab'
if any([key.lower().startswith('github') for key in os.environ]):
    ENV = 'github_action'

DOMAIN = ""
if "USERDOMAIN" in os.environ:
    DOMAIN = os.environ["USERDOMAIN"]

# ROOT DIRECTORY
ROOT = "https://raw.githubusercontent.com/labwons/pages/main/"
if not ENV == 'google_colab':
    ROOT = os.path.dirname(__file__)
    while not ROOT.endswith('pages'):
        ROOT = os.path.dirname(ROOT)

# DEPLOYMENT DIRECTORY
DOCS   = os.path.join(ROOT, r'docs')

# RESOURCE FILE DIRECTORIES
FILE = dDict()
FILE.AFTER_MARKET       = os.path.join(ROOT, r'src/fetch/market/parquet/aftermarket.parquet')
FILE.ANNUAL_STATEMENT   = os.path.join(ROOT, r'src/fetch/market/parquet/annualstatement.parquet')
FILE.QUARTER_STATEMENT  = os.path.join(ROOT, r'src/fetch/market/parquet/quarterstatement.parquet')
FILE.STATEMENT_OVERVIEW = os.path.join(ROOT, r'src/fetch/market/parquet/statementoverview.parquet')
FILE.SECTOR_COMPOSITION = os.path.join(ROOT, r'src/fetch/market/parquet/sectorcomposition.parquet')
FILE.BASELINE           = os.path.join(ROOT, r'src/fetch/market/parquet/baseline.parquet')
FILE.MACRO_BASELINE     = os.path.join(ROOT, r'src/fetch/macro/parquet/baseline.parquet')
FILE.ECOS               = os.path.join(ROOT, r'src/fetch/macro/parquet/ecos.parquet')
FILE.FRED               = os.path.join(ROOT, r'src/fetch/macro/parquet/fred.parquet')
FILE.PRICE              = os.path.join(ROOT, r'src/fetch/stock/parquet/price.parquet')

# RESOURCE DEPLOY DELIVERABLES
HTML = dDict()
HTML.MAP        = os.path.join(ROOT, r'docs/index.html')
HTML.BUBBLE     = os.path.join(ROOT, r'docs/bubble/index.html')
HTML.MACRO      = os.path.join(ROOT, r'docs/macro/index.html')

# RESOURCE PATH DIRECTORIES
PATH = dDict()
PATH.DOCS = os.path.join(ROOT, r'docs')
PATH.TEMPLATES = os.path.join(ROOT, r'src/build/apps/templates')
if ENV == "local":
    PATH.DESKTOP = os.path.join(os.environ['USERPROFILE'], 'Desktop')
    PATH.DOWNLOADS = os.path.join(os.environ['USERPROFILE'], 'Downloads')
    PATH.STUB = os.path.join(PATH.DOWNLOADS, 'labwons')

# GITHUB ENVIRONMENT PARAMETERS
GITHUB = dDict()
GITHUB.EVENT = os.environ.get("GITHUB_EVENT_NAME", "local")
GITHUB.CONFIG = dDict(
    AFTERMARKET = False,
    STATEMENT = False,
    SECTOR = False,
    ECOS = False,
    FRED = False,
    STOCKPRICE = False,
    STOCKDEPLOY = True,
)
def __RESET__():
    for key, val in GITHUB.CONFIG:
        if not key == "RESET":
            GITHUB.CONFIG[key] = False
GITHUB.CONFIG.RESET = __RESET__



# if __name__ == "__main__":

    # print(CLOCK())
    # print(ENV)
    # print(FILE.BASELINE)
    # print(FILE.GROUP)
    # print(FILE.ANNUAL_STATEMENT)
    # print(GITHUB.CONFIG)
    # GITHUB.CONFIG.RESET()
    # print(GITHUB.CONFIG)
    # GITHUB.CONFIG.ECOS = GITHUB.CONFIG.STATEMENT = True
    # print(GITHUB.CONFIG)

    # for key, value in os.environ.items():
    #     print(key, value)


### util.py

In [4]:
from numpy import isnan, nan
from pandas import isna
from typing import Union


def krw2currency(krw: int, limit:str='억') -> Union[str, float]:
    """
    KRW (원화) 입력 시 화폐 표기 법으로 변환(자동 계산)
    @krw 단위는 원 일 것
    """
    if isna(krw) or isnan(krw):
        return nan
    if krw >= 1e+12:
        krw /= 1e+8
        currency = f'{int(krw // 10000)}조'
        if int(krw % 10000):
            currency += f' {int(krw % 10000)}억'
        return currency
    if krw >= 1e+8:
        krw /= 1e+4
        currency = f'{int(krw // 10000)}억'
        if limit == '억':
            return currency
        if int(krw % 10000):
            currency += f' {int(krw % 10000)}만'
        return currency
    return f'{int(krw // 10000)}만'

def str2num(src: str) -> int or float:
    if isinstance(src, float):
        return src
    if src is None:
        return nan
    src = "".join([char for char in src if char.isdigit() or char == "."])
    if not src or src == ".":
        return nan
    if "." in src:
        return float(src)
    return int(src)

## fetch

### util.py

In [23]:
from bs4 import BeautifulSoup
from numpy import nan
from pandas import DataFrame, isna
from urllib.request import urlopen
from typing import Dict
import requests, json, pandas, warnings

warnings.filterwarnings("ignore")


class _web(object):

    def req(self, url:str):
        attr = f"_req_{url}_"
        if not hasattr(self, attr):
            req = requests.get(url, verify=False)
            if not req.status_code == 200:
                raise ConnectionError
            self.__setattr__(attr, req)
        return self.__getattribute__(attr)

    def html(self, url:str, parser:str="") -> BeautifulSoup:
        attr = f"_html_{url}_"
        if not hasattr(self, attr):
            parser = parser if parser else 'xml' if url.endswith('.xml') else 'lxml'
            self.__setattr__(attr, BeautifulSoup(self.req(url).text, parser))
        return self.__getattribute__(attr)

    def list(self, url:str, encoding:str='utf-8', displayed_only:bool=False) -> list:
        attr = f"_list_{url}_"
        if not hasattr(self, attr):
            encoding = "euc-kr" if "naver" in url else encoding
            self.__setattr__(attr, pandas.read_html(io=url, header=0, encoding=encoding, displayed_only=displayed_only))
        return self.__getattribute__(attr)

    def json(self, url:str) -> json:
        attr = f"_json_{url}_"
        if not hasattr(self, attr):
            data = json.loads(urlopen(url=url).read().decode('utf-8-sig', 'replace').replace(" ", "").replace("\t", ""))
            self.__setattr__(attr, data)
        return self.__getattribute__(attr)

    def data(self, url:str, key:str=""):
        if url.endswith('.json'):
            return pandas.DataFrame(self.json(url)[key] if key else self.json(url))
        elif url.endswith('.csv'):
            return pandas.read_csv(url, encoding='utf-8')
        elif url.endswith('.pkl'):
            return pandas.read_pickle(url)
        else:
            raise KeyError(f"Unknown data type: {url}")

class multiframes(DataFrame):

    __mem__ = {}
    def __init__(self, frames:Dict[str, DataFrame]):
        base = list(frames.values())[0]
        self.__mem__ = frames.copy()
        super().__init__(data=base.values, index=base.index, columns=base.columns)
        return

    def __getattr__(self, item):
        if item in self.__mem__:
            return self.__mem__[item]
        return super().__getattr__(name=item)


def str2num(src: str) -> int or float:
    if isinstance(src, float):
        return src
    src = "".join([char for char in src if char.isdigit() or char == "."])
    if not src or src == ".":
        return nan
    if "." in src:
        return float(src)
    return int(src)

def cutString(string:str, deleter:list) -> str:
    _deleter = deleter.copy()
    while _deleter:
        string = string.replace(_deleter.pop(0), '')
    return string

def krwFormat(krw: int) -> str:
    if krw is nan or isna(krw):
        return krw
    zo, euk = int(krw // 10000), int(krw % 10000)
    return f'{zo}조 {euk}억' if zo else f'{euk}억'


# Alias
web = _web()

### market

#### finances.py

In [ ]:
from pandas import (
    concat,
    DataFrame,
    read_json,
    Series
)
from re import DOTALL, sub
from requests import get
from time import time
from typing import Any, List, Union
from xml.etree.ElementTree import Element, fromstring

if not "FILE" in globals():
    try:
        from ...common.env import FILE
    except ImportError:
        from src.common.env import FILE



class FinancialStatement:
    _log: List[str] = []

    def __init__(self, update:bool=False, *tickers):
        if not update:
            return

        stime = time()
        if update and tickers:
            self.tickers = list(tickers)
            self.log = f'RUN [Build Numbers Cache] PARTIAL UPDATE N={len(tickers)}'
        else:
            self.log = f'RUN [Build Numbers Cache] FULL UPDATE'
            baseline = read_json(FILE.BASELINE, orient='index')
            baseline.index = baseline.index.astype(str).str.zfill(6)
            self.tickers = baseline.index

        overview, annual, quarter = [], {}, {}
        for ticker in self.tickers:
            xml = self.fetch(ticker, debug=False)
            if xml is None:
                self.log = f'... Empty xml or Failed to fetch: {ticker}'
                continue
            overview.append(self.numbers(xml, name=ticker))
            annual[ticker] = self.annualStatement(xml)
            quarter[ticker] = self.quarterStatement(xml)

        self.overview = concat(overview, axis=1)
        self.annual = concat(annual, axis=1)
        self.quarter = concat(quarter, axis=1)

        self.log = f'END [Build Numbers Cache] {len(self):,d} Stocks / Elapsed: {time() - stime:.2f}s'
        return

    def __len__(self):
        return len(self.tickers)

    @property
    def log(self) -> str:
        return "\n".join(self._log)

    @log.setter
    def log(self, log: str):
        self._log.append(log)

    @classmethod
    def _statement(cls, xml:Element, tag: str) -> DataFrame:
        obj = xml.find(tag)
        if obj is None:
            return DataFrame()
        columns = [val.text for val in obj.findall('field')]
        index, data = [], []
        for record in obj.findall('record'):
            index.append(record.find('date').text)
            data.append([val.text for val in record.findall('value')])
        return DataFrame(index=index, columns=columns, data=data)

    @classmethod
    def fetch(cls, ticker: str, debug: bool = False) -> Union[Any, Element]:
        try:
            resp = get(url=f"http://cdn.fnguide.com/SVO2/xml/Snapshot_all/{ticker}.xml")
            resp.encoding = 'euc-kr'
            text = resp.text.replace("<![CDATA[", "").replace("]]>", "")
            text = sub(r'<business_summary>.*?</business_summary>', '', text, flags=DOTALL)
            if debug:
                print(text)
            return fromstring(text)
        except Exception as reason:
            cls._log.append(f'... Failed to fetch: {ticker} / {reason}')
        return

    @classmethod
    def numbers(cls, ticker_or_xml: Union[str, Element], name: str = None) -> Series:
        xml = cls.fetch(ticker_or_xml) if isinstance(ticker_or_xml, str) else ticker_or_xml
        obj = {child.tag: child.text for child in xml.find('price')}
        if xml.find('consensus') is not None:
            obj.update({child.tag: child.text for child in xml.find('consensus')})
        return Series(obj, name=name)

    @classmethod
    def annualStatement(cls, ticker_or_xml: Union[str, Element]) -> DataFrame:
        xml = cls.fetch(ticker_or_xml) if isinstance(ticker_or_xml, str) else ticker_or_xml
        separate = cls._statement(xml, 'financial_highlight_ifrs_B/financial_highlight_annual')
        consolidate = cls._statement(xml, 'financial_highlight_ifrs_D/financial_highlight_annual')
        return concat({'별도': separate, '연결': consolidate}, axis=1)

    @classmethod
    def quarterStatement(cls, ticker_or_xml: Union[str, Element]) -> DataFrame:
        xml = cls.fetch(ticker_or_xml) if isinstance(ticker_or_xml, str) else ticker_or_xml
        separate = cls._statement(xml, 'financial_highlight_ifrs_B/financial_highlight_quarter')
        consolidate = cls._statement(xml, 'financial_highlight_ifrs_D/financial_highlight_quarter')
        return concat({'별도': separate, '연결': consolidate}, axis=1)


if __name__ == "__main__":
    fs = FinancialStatement(update=True)
    print(fs.log)


RUN [Build Numbers Cache] FULL UPDATE
END [Build Numbers Cache] 1,323 Stocks / Elapsed: 611.23s


#### aftermarket.py

In [ ]:
from datetime import datetime, timedelta
from io import StringIO
from pandas import (
    concat,
    DataFrame,
    read_html,
    set_option,
    Series
)
from pykrx.stock import (
    get_exhaustion_rates_of_foreign_investment,
    get_nearest_business_day_in_a_week,
    get_market_cap_by_ticker,
    get_market_fundamental,
    get_market_ohlcv_by_date,
    get_market_ticker_list
)
from requests import get
from time import time
from typing import Dict, Iterable, List

set_option('future.no_silent_downcasting', True)

INTERVALS: Dict[str, int] = {
    'D0': 0, 'return1Day': 1, 'return1Week': 7,
    'return1Month': 30, 'return3Month': 91, 'return6Month': 182, 'return1Year': 365
}


class AfterMarket:
    _log: List[str] = []

    def __init__(self, update: bool = False):
        if not update:
            return

        stime = time()
        self.log = f'RUN [AFTER MARKET]'
        date = get_nearest_business_day_in_a_week()
        self.tradingDate = datetime.strptime(date, "%Y%m%d").strftime("%Y/%m/%d")

        try:
            marketCap = get_market_cap_by_ticker(date=date, market='ALL', alternative=True)
            self.log = f'... {"Failed" if marketCap.empty else "Success"} fetching market cap'
        except Exception as reason:
            marketCap = DataFrame()
            self.log = f'... Failed fetching market cap: {reason}'

        try:
            multiples = get_market_fundamental(date=date, market='ALL', alternative=True)
            self.log = f'... {"Failed" if multiples.empty else "Success"} fetching multiples'
        except Exception as reason:
            multiples = DataFrame()
            self.log = f'... Failed fetching multiples: {reason}'

        try:
            foreignRate = get_exhaustion_rates_of_foreign_investment(date=date, market='ALL')
            self.log = f'... {"Failed" if foreignRate.empty else "Success"} fetching foreign rate'
        except Exception as reason:
            foreignRate = DataFrame()
            self.log = f'... Failed fetching foreign rate: {reason}'

        try:
            ipo = read_html(
                io=StringIO(get('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download').text),
                encoding='euc-kr'
            )[0].set_index(keys='종목코드')
            ipo.index = ipo.index.astype(str).str.zfill(6)
            self.log = f'... {"Failed" if ipo.empty else "Success"} fetching ipo list'
        except Exception as reason:
            ipo = DataFrame()
            self.log = f'... Failed fetching ipo list: {reason}'

        try:
            ks = Series(index=get_market_ticker_list(date=date, market='KOSPI')).fillna('KOSPI')
            kq = Series(index=get_market_ticker_list(date=date, market='KOSDAQ')).fillna('KOSDAQ')
            marketType = concat([ks, kq], axis=0)
            marketType.name = "market"
            self.log = f'... {"Failed" if marketType.empty else "Success"} fetching market type'
        except Exception as reason:
            marketType = DataFrame()
            self.log = f'... Failed fetching market type: {reason}'

        merged = concat([marketCap, multiples, foreignRate], axis=1)
        c_active_ipo = merged.index.isin(ipo.index)
        c_no_konex = ~merged.index.isin(get_market_cap_by_ticker(date=date, market='KONEX').index)
        c_active_trade = merged['거래량'] > 0
        c_market_cap = merged['시가총액'] >= merged['시가총액'].median()

        merged = merged[c_active_ipo & c_no_konex & c_active_trade & c_market_cap]
        merged = merged.join(marketType, how='left')
        merged.index.name = 'ticker'

        try:
            returns = self.fetchReturns(date, merged.index)
            merged = merged.join(returns, how='left')
            self.log = f'... {"Failed" if returns.empty else "Success"} fetching returns'
        except Exception as reason:
            self.log = f'... Failed fetching returns: {reason}'
        self.data = merged = merged.sort_values(by='시가총액', ascending=False)

        self.log = f'End [AFTER MARKET] / {len(merged)} stocks / Elapsed: {time() - stime:.2f}s'
        return

    @property
    def log(self) -> str:
        return "\n".join(self._log)

    @log.setter
    def log(self, log: str):
        self._log.append(log)

    @classmethod
    def fetchReturns(cls, date: str, tickers: Iterable = None) -> DataFrame:
        tdate = datetime.strptime(date, "%Y%m%d")
        intv, objs = {}, {}
        for key, val in INTERVALS.items():
            fdate = (tdate - timedelta(val)).strftime("%Y%m%d")
            intv[key] = dt = get_nearest_business_day_in_a_week(fdate)
            objs[key] = get_market_cap_by_ticker(date=dt, market='ALL', alternative=True)

        base = concat(objs, axis=1)
        base = base[base.index.isin(tickers)]
        returns = concat({
            dt: base['D0']['종가'] / base[dt]['종가'] - 1 for dt in objs
        }, axis=1)
        returns.drop(columns=['D0'], inplace=True)

        diff = base[base['return1Year']['상장주식수'] != base['D0']['상장주식수']].index
        fdate = (tdate - timedelta(380)).strftime("%Y%m%d")

        ohlc = concat({
            ticker: get_market_ohlcv_by_date(fromdate=fdate, todate=date, ticker=ticker)['종가']
            for ticker in diff
        }, axis=1)

        objs = {}
        for interval in returns.columns:
            ohlc_copy = ohlc[ohlc.index >= intv[interval]]
            _returns = ohlc_copy.iloc[-1] / ohlc_copy.iloc[0] - 1
            objs[interval] = _returns
        returns.update(concat(objs=objs, axis=1))
        return round(100 * returns, 2)


if __name__ == "__main__":
    afterMarket = AfterMarket(update=True)

    print(afterMarket.log)



RUN [AFTER MARKET]
... Success fetching market cap
... Success fetching multiples
... Success fetching foreign rate
... Success fetching ipo list
... Failed fetching market type: name 'marketType' is not defined
... Success fetching returns
End [AFTER MARKET] / 1384 stocks / Elapsed: 233.55s


#### sector.py

In [ ]:
from pandas import DataFrame, concat
from re import compile
from requests import get, Session
from time import sleep, time
from typing import Dict, List


SECTOR_CODE:Dict[str, str] = {
    'WI100': '에너지', 'WI110': '화학',
    'WI200': '비철금속', 'WI210': '철강', 'WI220': '건설', 'WI230': '기계', 'WI240': '조선', 'WI250': '상사,자본재', 'WI260': '운송',
    'WI300': '자동차', 'WI310': '화장품,의류', 'WI320': '호텔,레저', 'WI330': '미디어,교육', 'WI340': '소매(유통)',
    'WI400': '필수소비재', 'WI410': '건강관리',
    'WI500': '은행', 'WI510': '증권', 'WI520': '보험',
    'WI600': '소프트웨어', 'WI610': 'IT하드웨어', 'WI620': '반도체', 'WI630': 'IT가전', 'WI640': '디스플레이',
    'WI700': '통신서비스',
    'WI800': '유틸리티'
}

CODE_LABEL:Dict[str, str] = {
    'CMP_CD': 'ticker', 'CMP_KOR': 'name',
    'SEC_CD': 'sectorCode', 'SEC_NM_KOR': 'sectorName',
    'IDX_CD': 'industryCode', 'IDX_NM_KOR': 'industryName',
}

REITS_CODE:Dict[str, str] = {
    "088980": "맥쿼리인프라",
    "395400": "SK리츠",
    "365550": "ESR켄달스퀘어리츠",
    "330590": "롯데리츠",
    "348950": "제이알글로벌리츠",
    "293940": "신한알파리츠",
    "432320": "KB스타리츠",
    "094800": "맵스리얼티1",
    "357120": "코람코라이프인프라리츠",
    "448730": "삼성FN리츠",
    "451800": "한화리츠",
    "088260": "이리츠코크렙",
    "334890": "이지스밸류리츠",
    "377190": "디앤디플랫폼리츠",
    "404990": "신한서부티엔디리츠",
    "417310": "코람코더원리츠",
    "400760": "NH올원리츠",
    "350520": "이지스레지던스리츠",
    "415640": "KB발해인프라",
}

EXCEPTIONALS = {
    '950160': {
        "name": "코오롱티슈진",
        "industryCode": "WI410",
        "industryName": "건강관리",
        "sectorCode": "G35",
        "sectorName": "건강관리"
    },
    '950210': {
        'name': '프레스티지바이오파마',
        "industryCode": "WI410",
        "industryName": "건강관리",
        "sectorCode": "G35",
        "sectorName": "건강관리"
    },
    '009410': {
        'name': '태영건설',
        "industryCode": "WI220",
        "industryName": "건설",
        "sectorCode": "G20",
        "sectorName": "산업재"
    },
    '052020': {
        'name': '에스티큐브',
        "industryCode": "WI410",
        "industryName": "건강관리",
        "sectorCode": "G35",
        "sectorName": "건강관리"
    }
}

class SectorComposition:

    _log:List[str] = []
    state:str = "SUCCESS"
    def __init__(self):
        stime = time()

        self.log = f'RUN [Update Sector Composition]'
        try:
            date = compile(r"var\s+dt\s*=\s*'(\d{8})'") \
                   .search(get('https://www.wiseindex.com/Index/Index#/G1010.0.Components').text) \
                   .group(1)
        except Exception as reason:
            self.log = f'- {reason}'
            self.log = f'END [Update Sector Composition] / Elapsed: {time() - stime:.2f}s'
            self.state = "FAILED"
            return

        objs, size = [], len(SECTOR_CODE) + 1
        for n, (code, name) in enumerate(SECTOR_CODE.items()):
            self.log = f"... {str(n + 1).zfill(2)} / {size} : {code} {name} :: "
            objs.append(self.fetchWiseGroup(code, date))

        reits = DataFrame(data={'CMP_KOR': REITS_CODE.values(), 'CMP_CD':REITS_CODE.keys()})
        reits[['SEC_CD', 'IDX_CD', 'SEC_NM_KOR', 'IDX_NM_KOR']] = ['G99', 'WI999', '리츠', '리츠']
        objs.append(reits)
        self.log = f"... {size} / {size} : WI999 리츠 :: SUCCESS"

        data = concat(objs, axis=0, ignore_index=True)

        data.drop(inplace=True, columns=[key for key in data if not key in CODE_LABEL])
        data.drop(inplace=True, index=data[data['SEC_CD'].isna()].index)
        data.rename(inplace=True, columns=CODE_LABEL)
        data.set_index(inplace=True, keys="ticker")
        data['industryName'] = data['industryName'].str.replace("WI26 ", "")

        sc_mdi = data[(data['industryCode'] == 'WI330') & (data['sectorCode'] == 'G50')].index
        sc_edu = data[(data['industryCode'] == 'WI330') & (data['sectorCode'] == 'G25')].index
        sc_sw = data[(data['industryCode'] == 'WI600') & (data['sectorCode'] == 'G50')].index
        sc_it = data[(data['industryCode'] == 'WI600') & (data['sectorCode'] == 'G45')].index
        data.loc[sc_mdi, 'industryCode'], data.loc[sc_mdi, 'industryName'] = 'WI331', '미디어'
        data.loc[sc_edu, 'industryCode'], data.loc[sc_edu, 'industryName'] = 'WI332', '교육'
        data.loc[sc_sw, 'industryCode'], data.loc[sc_sw, 'industryName'] = 'WI601', '소프트웨어'
        data.loc[sc_it, 'industryCode'], data.loc[sc_it, 'industryName'] = 'WI602', 'IT서비스'

        adder = {}
        for key in EXCEPTIONALS:
            if not key in data.index:
                adder[key] = EXCEPTIONALS[key]
        exceptionals = DataFrame(adder).T
        self.data = concat(objs=[data, exceptionals], axis=0)
        self.data['date'] = date
        self.log = f'END [Update Sector Composition] / {len(data)} Stocks / Elapsed: {time() - stime:.2f}s'
        if "FAIL" in self.log:
            self.state = "FAILED"
        return

    @property
    def log(self) -> str:
        return "\n".join(self._log)

    @log.setter
    def log(self, log:str):
        self._log.append(log)

    @classmethod
    def fetchWiseGroup(cls, code:str, date:str="", countdown:int=5) -> DataFrame:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36 Edg/133.0.0.0",
            "Accept-Language": "ko,en;q=0.9,en-US;q=0.8",
            "Referer": "http://www.wiseindex.com/"
        }

        session = Session()
        session.headers.update(headers)
        try:
            resp = get(
                url=f'http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt={date}&sec_cd={code}',
                # proxies=proxies
            )
        except Exception as reason:
            cls._log[-1] += "FAILED: "
            cls._log.append(f'-  {reason}')
            return DataFrame()

        if not resp.status_code == 200:
            if countdown == 0:
                cls._log[-1] += "FAILED: "
                cls._log.append(f'- response status: {resp.status_code} for {code} / {SECTOR_CODE[code]}')
                return DataFrame()
            else:
                sleep(5)
                return cls.fetchWiseGroup(code, date, countdown - 1)
        if "hmg-corp" in resp.text:
            cls._log[-1] += "FAILED: BLOCKED"
            return DataFrame()
        cls._log[-1] += "SUCCESS"
        return DataFrame(resp.json()['list'])

if __name__ == "__main__":
    sector = SectorComposition()
    print(sector.state)
    print(sector.log)


SUCCESS
RUN [Update Sector Composition]
... 01 / 27 : WI100 에너지 :: SUCCESS
... 02 / 27 : WI110 화학 :: SUCCESS
... 03 / 27 : WI200 비철금속 :: SUCCESS
... 04 / 27 : WI210 철강 :: SUCCESS
... 05 / 27 : WI220 건설 :: SUCCESS
... 06 / 27 : WI230 기계 :: SUCCESS
... 07 / 27 : WI240 조선 :: SUCCESS
... 08 / 27 : WI250 상사,자본재 :: SUCCESS
... 09 / 27 : WI260 운송 :: SUCCESS
... 10 / 27 : WI300 자동차 :: SUCCESS
... 11 / 27 : WI310 화장품,의류 :: SUCCESS
... 12 / 27 : WI320 호텔,레저 :: SUCCESS
... 13 / 27 : WI330 미디어,교육 :: SUCCESS
... 14 / 27 : WI340 소매(유통) :: SUCCESS
... 15 / 27 : WI400 필수소비재 :: SUCCESS
... 16 / 27 : WI410 건강관리 :: SUCCESS
... 17 / 27 : WI500 은행 :: SUCCESS
... 18 / 27 : WI510 증권 :: SUCCESS
... 19 / 27 : WI520 보험 :: SUCCESS
... 20 / 27 : WI600 소프트웨어 :: SUCCESS
... 21 / 27 : WI610 IT하드웨어 :: SUCCESS
... 22 / 27 : WI620 반도체 :: SUCCESS
... 23 / 27 : WI630 IT가전 :: SUCCESS
... 24 / 27 : WI640 디스플레이 :: SUCCESS
... 25 / 27 : WI700 통신서비스 :: SUCCESS
... 26 / 27 : WI800 유틸리티 :: SUCCESS
... 27 / 27 : WI999 리츠 :: SUCC

In [ ]:
from google.colab import files
sector.data.to_parquet("sectorcomposition.parquet", engine='pyarrow')
files.download("sectorcomposition.parquet")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### stock


#### krx.py

In [ ]:
from datetime import datetime, timedelta
from pandas import concat, Series, DataFrame
from pykrx.stock import get_market_ohlcv_by_date, get_market_cap_by_date


class PyKrx:
    """
    Fetch source data from PyKrx (through <package; pyPyKrx>)

    @ohlcv
        constraint  : common
        type        : DataFrame
        description : stock price (open, high, low, close) and volume
        columns     : ["open", "high", "low", "close", "volume"]
        example     :
                         open   high    low  close    volume
            date
            2013-12-13  28200  28220  27800  27800    201065
            2013-12-16  27820  28080  27660  28000    179088
            2013-12-17  28340  28340  27860  27900    155248
            ...           ...    ...    ...    ...       ...
            2023-12-07  71800  71900  71100  71500   8862017
            2023-12-08  72100  72800  71900  72600  10859463
            2023-12-11  72800  73000  72200  73000   9406504

    @quarterlyMarketCap
        constraint  : stock only
        type        : Series
        description : quarterly market cap
        example     :
              month
            2019/03     93522
            2019/06     95563
            2019/09     89922
                ...       ...
            2023/03     83071
            2023/06     85838
            2023/09     93241
            2023/11     95497
            Name: marketCap, dtype: int32
    """

    def __init__(self, ticker:str, period:int=10, freq:str="d"):
        self.ticker = ticker
        self.period = period
        self.freq = freq
        return

    def getMarketCap(self) -> DataFrame:
        if not hasattr(self, "__cap"):
            cap = get_market_cap_by_date(
                fromdate=(datetime.today() - timedelta(365 * 8)).strftime("%Y%m%d"),
                todate=datetime.today().strftime("%Y%m%d"),
                freq='m',
                ticker=self.ticker
            )
            self.__setattr__("__cap", cap)
        return self.__getattribute__("__cap")

    @classmethod
    def get_multi_ohlcv(cls, tickers: list, period: int = 5, freq: str = 'd') -> DataFrame:
        todate = datetime.today().strftime("%Y%m%d")
        frdate = (datetime.today() - timedelta(365 * period)).strftime("%Y%m%d")
        objs = {}
        for tic in tickers:
            ohlcv = get_market_ohlcv_by_date(
                fromdate=frdate,
                todate=todate,
                ticker=tic,
                freq=freq
            )

            trade_stop = ohlcv[ohlcv.시가 == 0].copy()
            if not trade_stop.empty:
                ohlcv.loc[trade_stop.index, ['시가', '고가', '저가']] = trade_stop.종가
            ohlcv.index.name = 'date'
            objs[tic] = ohlcv.rename(columns=dict(시가='open', 고가='high', 저가='low', 종가='close', 거래량='volume'))
        return concat(objs, axis=1)

    @property
    def ohlcv(self) -> DataFrame:
        todate = datetime.today().strftime("%Y%m%d")
        frdate = (datetime.today() - timedelta(365 * self.period)).strftime("%Y%m%d")
        ohlcv = get_market_ohlcv_by_date(
            fromdate=frdate,
            todate=todate,
            ticker=self.ticker,
            freq=self.freq
        )

        trade_stop = ohlcv[ohlcv.시가 == 0].copy()
        if not trade_stop.empty:
            ohlcv.loc[trade_stop.index, ['시가', '고가', '저가']] = trade_stop.종가
        ohlcv.index.name = 'date'
        return ohlcv.rename(columns=dict(시가='open', 고가='high', 저가='low', 종가='close', 거래량='volume'))[[
            'open', 'high', 'low', 'close', 'volume'
        ]]

    @property
    def quarterlyMarketCap(self) -> Series:
        cap = self.getMarketCap()
        cap = cap[
            cap.index.astype(str).str.contains('03') | \
            cap.index.astype(str).str.contains('06') | \
            cap.index.astype(str).str.contains('09') | \
            cap.index.astype(str).str.contains('12') | \
            (cap.index == cap.index[-1])
        ]
        cap.index = cap.index.strftime("%Y/%m")
        cap.index = [
            col.replace("03", "1Q").replace("06", "2Q").replace("09", "3Q").replace("12", "4Q") for col in cap.index
        ]
        cap.index.name = "quarter"
        return Series(index=cap.index, data=cap['시가총액'] / 100000000, dtype=int)

    @property
    def yearlyMarketCap(self) -> Series:
        cap = self.getMarketCap()
        cap = cap[cap.index.astype(str).str.contains('12') | (cap.index == cap.index[-1])]
        cap.index = cap.index.strftime("%Y/%m")
        cap.index.name = "year"
        return Series(index=cap.index, data=cap['시가총액'] / 100000000, dtype=int)



if __name__ == "__main__":
    from pandas import set_option
    set_option('display.expand_frame_repr', False)

    pyKrx = PyKrx(
        "005930"
        # "069500"
    )
    print(PyKrx.ohlcv)
    print(PyKrx.quarterlyMarketCap)


#### fnguide.py

In [36]:
# try:
#     from ..util import web, str2num, cutString, multiframes
# except ImportError:
#     from src.fetch.util import web, str2num, cutString, multiframes
from numpy import nan
from pandas import concat, DataFrame, Series, to_datetime


class _url:
    """
    Set of urls for fnguide pages
    """

    class _cdn:

        def __init__(self, ticker: str, gb: str):
            base = "http://cdn.fnguide.com/SVO2/json/chart"
            self.products = f"{base}/02/chart_A{ticker}_01_N.json"
            self.multipleBands = f"{base}/01_06/chart_A{ticker}_D.json"
            self.expenses = f"{base}/02/chart_A{ticker}_D.json"
            self.profitConsensusAnnual = f"{base}/07_01/chart_A{ticker}_{gb}_A.json"
            self.profitConsensusQuarter = f"{base}/07_01/chart_A{ticker}_{gb}_Q.json"
            self.priceConsensus = f"{base}/01_02/chart_A{ticker}.json"
            self.abstractConsensusRelevant = f"{base}/07_02/chart_A{ticker}_{gb}_FY1.json"
            self.abstractConsensusForward = f"{base}/07_02/chart_A{ticker}_{gb}_FY2.json"
            self.foreignRate3Months = f"{base}/01_01/chart_A{ticker}_3M.json"
            self.foreignRate1Year = f"{base}/01_01/chart_A{ticker}_1Y.json"
            self.foreignRate3Years = f"{base}/01_01/chart_A{ticker}_3Y.json"
            self.benchmarkMultiples = f"{base}/01_04/chart_A{ticker}_{gb}.json"
            self.shares = f"{base}/08_01/chart_A{ticker}.json"
            self.shortSell = f"{base}/11_01/chart_A{ticker}_SELL1Y.json"
            self.shortBalance = f"{base}/11_01/chart_A{ticker}_BALANCE1Y.json"
            return

    def __init__(self, ticker:str):
        self.ticker = ticker
        self.xml = f"http://cdn.fnguide.com/SVO2/xml/Snapshot_all/{ticker}.xml"
        return

    def __html__(self, assetType:str, page:str, ReportGB:str, stkGb:str) -> str:
        return f"http://comp.fnguide.com/SVO2/ASP/{assetType}_{page}.asp?" \
               f"pGB=1&" \
               f"gicode=A{self.ticker}&" \
               f"cID=&" \
               f"MenuYn=Y" \
               f"&ReportGB={ReportGB}" \
               f"&NewMenuID=" \
               f"&stkGb={stkGb}"

    @property
    def gb(self) -> str:
        try:
            tbs = web.list(self.snapshot)
            return "B" if tbs[11].iloc[1].isnull().sum() > tbs[14].iloc[1].isnull().sum() else "D"
        except IndexError:
            return "D"

    @property
    def cdn(self):
        return self._cdn(self.ticker, self.gb)

    @property
    def snapshot(self) -> str:
        return self.__html__("SVD", "Main", "", "701")

    @property
    def corp(self) -> str:
        return self.__html__("SVD", "Corp", self.gb, "701")

    @property
    def finance(self) -> str:
        return self.__html__("SVD", "Finance", self.gb, "701")

    @property
    def ratio(self) -> str:
        return self.__html__("SVD", "FinanceRatio", self.gb, "701")

    @property
    def invest(self) -> str:
        return self.__html__("SVD", "Invest", "", "701")



class fnguide:
    """
    Fetch source data from fnguide

    @abstract
        constraint  : property
        type        : DataFrame (multi frames included)
        description : abstracted dataframe of financial statement, ratio and others.
        columns     : ['이자수익', '영업이익', '영업이익(발표기준)', '당기순이익',
                       '지배주주순이익', '비지배주주순이익',
                       '자산총계', '부채총계', '자본총계', '지배주주지분', '비지배주주지분',
                       '자본금', '부채비율', '유보율', '영업이익률',
                       '지배주주순이익률', 'ROA', 'ROE', 'EPS(원)', 'BPS(원)', 'DPS(원)', 'PER', 'PBR',
                       '발행주식수', '배당수익률']
        example     :
                      이자수익 영업이익 당기순이익 ...   PER   PBR 발행주식수 배당수익률
            기말
            2018/12        NaN     NaN        NaN  ...   NaN   NaN       NaN        NaN
            2019/12     105768   28000      20376  ...  4.29  0.39    722268       6.03
            2020/12      95239   20804      15152  ...  5.38  0.30    722268       3.70
            2021/12      98947   36597      28074  ...  3.56  0.36    728061       7.09
            2022/12     146545   44305      33240  ...  2.68  0.29    728061       9.78
            2023/12(E)  198704   40045      30132  ...  3.25  0.30       NaN        NaN

    @benchmarkMultiples
        constraint  : property
        type        : DataFrame (Multi-Indexed)
        description : compared data of multiples: ["PER", "EV/EBITDA", "ROE", "배당수익률"]
                      also, by index.
        columns     : MultiIndex([(       'PER',  '우리금융지주'),
                                  (       'PER', '코스피 금융업'),
                                  (       'PER',     '코스피'),
                                  ( 'EV/EBITDA',  '우리금융지주'),
                                  ( 'EV/EBITDA', '코스피 금융업'),
                                  ( 'EV/EBITDA',     '코스피'),
                                  (       'ROE',  '우리금융지주'),
                                  (       'ROE', '코스피 금융업'),
                                  (       'ROE',     '코스피'),
                                  ('배당수익률',  '우리금융지주'),
                                  ('배당수익률', '코스피 금융업'),
                                  ('배당수익률',     '코스피')
                                ])
        example     :
                                                  PER  ...                           배당수익률
                  우리금융지주  코스피 금융업  코스피  ...  우리금융지주  코스피 금융업  코스피
            2021          3.56           5.94   11.08  ...          7.09           3.48    1.78
            2022          2.68           5.53   10.87  ...          9.78           4.32    2.22
            2023E         3.25           6.08   17.54  ...           NaN            NaN     NaN

    @businessSummary
        constraint  : property
        type        : str
        description : business summary of the corp.
        example     :
            한국 및 DX부문 해외 9개 지역총괄과 DS부문 해외 5개 지역총괄, SDC, Harman 등 233개의
            종속기업으로 구성된 글로벌 전자기업임. 세트사업은 TV를 비롯 모니터, 냉장고, 세탁기,
            에어컨, 스마트폰, 네트워크시스템, 컴퓨터 등을 생산하는 DX부문이 있음.
            부품 사업에는 DRAM, NAND Flash, 모바일AP 등의 제품을 생산하고 있는 DS 부문과 중소형OLED
            등의 디스플레이 패널을 생산하고 있는 SDC가 있음.

            3분기에는 스마트폰 플래그십 신제품 출시와 디스플레이 프리미엄 제품 판매 확대로 견조한 실적을
            거둔 디스플레이와 MX(모바일경험)가 반도체 부문의 영업손실을 상쇄함. 메모리 반도체의 영업적자는
            직전분기 대비 판매단가가 상승하며 축소됐으나, 시스템 LSI 및 파운드리의 영업적자는 부진한
            레거시 파운드리 가동률로 소폭 확대됨. 4분기는 메모리 부문 적자 축소와 디스플레이의 북미
            고객사 신제품 효과가 지속되며 실적 개선이 예상됨.

    @cashFlow
        constraint  : property
        type        : DataFrame
        description : cash flow of the corp.
        example     :
                     영업현금흐름  투자현금흐름  재무현금흐름  환율변동손익  현금및현금성자산
            2020/12        123146       -118404          2521          -563             29760
            2021/12        197976       -223923         44923          1843             50580
            2022/12        147805       -178837         28218          2005             49770
            2023/2Q         -6940        -53509         70579           508             60408

    @consensusOutstanding
        constraint  : property
        type        : Series
        description : abbreviated consensus
        example     :
            투자의견         4.0
            목표주가     15411.0
            EPS           3908.0
            PER              3.3
            추정기관수      18.0
            dtype: float64

    @consensusPrice
        constraint  : property
        type        : DataFrame
        description : average price consensus of trailing 1 year,
                      given if and only if more than 3 consensus data is gathered.
        example     :
                       투자의견  컨센서스     종가   격차
            날짜
            2022-11-23      4.0   16378.0  12300.0 -24.90
            2022-11-24      4.0   16378.0  12550.0 -23.37
            2022-11-25      4.0   16378.0  12450.0 -23.98
            ...             ...       ...      ...    ...
            2023-11-20      4.0   15411.0  12490.0 -18.95
            2023-11-21      4.0   15411.0  12720.0 -17.46
            2023-11-22      4.0   15411.0  12700.0 -17.59

    @consensusProfit
        constraint  : property
        type        : DataFrame (multi frames included)
        description : yearly or quarterly profit consensus
        example     :
            [Yearly: <attribute; Y>]
                    매출실적  매출전망  영업이익실적  영업이익전망
            기말
            2020/12  2013.35   2032.00        778.82        784.67
            2021/12  2801.67   2773.23       1171.04       1144.04
            2022/12  3224.23   3340.00       1366.35       1460.00
            2023/12      NaN   2578.67           NaN       1055.50
            2024/12      NaN   3048.83           NaN       1278.83
            2025/12      NaN   3469.00           NaN       1465.50

            [Quarterly: <attribute; Q>]
                    매출실적  매출전망  영업이익실적  영업이익전망
            기말
            2023/03   490.91    732.25        172.63         277.5
            2023/06   751.31    696.25        335.62         266.0
            2023/09   733.99    770.25        333.24         332.0
            2023/12      NaN    601.60           NaN         213.0
            2024/03      NaN    582.50           NaN         219.5
            2024/06      NaN    817.00           NaN         366.5

    @consensusThisFiscalYear, consensusNextFiscalYear
        constraint  : property
        type        : DataFrame
        description : abstracted consensus data
        columns     : ['매출', '매출(최대)', '매출(최소)', '영업이익', '영업이익(최대)', '영업이익(최소)', 'EPS',
                       'EPS(최대)', 'EPS(최소)', 'PER', 'PER(최대)', 'PER(최소)', '12M PER']
        example     :
                           매출  매출(최대)  매출(최소)   영업이익  영업이익(최대) 영업이익(최소)  ...  12M PER
            날짜
            2022/11   3063374.5     3288390     2826800     336985          419430         265250  ...     15.3
            2022/12  2942704.08     3173270     2635050     291990          389990         196600  ...    15.97
            2023/01  2820243.82     3073260     2635050  211293.59          342396         128930  ...    22.15
            2023/02  2728378.14     3073260     2581450  168233.05          329278          97490  ...    22.45
            2023/03   2723824.5     2900830     2594060  114761.09          184940          42540  ..     25.65
            2023/04  2688688.59     2884560     2560260     100754          184940          46570  ...    25.21
            2023/05  2678715.91     2884560     2540150   95985.36          122270          59390  ...    24.93
            2023/06  2660441.74     2785651     2476590   95079.48          122270          59390  ...    23.27
            2023/07  2604180.14     2708860     2527000   85640.81          126210          61590  ...    19.75
            2023/08  2609199.41     2708860     2527000   85829.45          126210          46620  ...       18
            2023/09   2613926.3     2708860     2527000   71636.43          100390          41620  ...    18.16
            2023/10  2609787.67     2661845     2528300   72144.57           96690          57010  ...    19.22

    @expenses
        constraint  : property
        type        : DataFrame (multi frames included)
        description : expenses ratio
        example     :
                     판관비율  매출원가율
            기말
            2019/12     16.58       20.62
            2020/12     13.81       12.31
            2021/12     15.25       10.70
            2022/12     10.69       14.06

    @financialStatement
        constraint  : property
        type        : DataFrame (multi frames included)
        description : financial statement of the prior (auto detected by linked or separated)
        columns     : ['자산', '유동자산', '재고자산', '유동생물자산', '유동금융자산',
                       '매출채권및기타유동채권', '당기법인세자산', '계약자산', '반품환불자산', '배출권',
                       '기타유동자산', '현금및현금성자산', '매각예정비유동자산및처분자산집단',
                       '비유동자산', '유형자산', '무형자산', '비유동생물자산', '투자부동산', '장기금융자산',
                       '관계기업등지분관련투자자산', '장기매출채권및기타비유동채권', '이연법인세자산',
                       '장기당기법인세자산', '계약자산', '반품환불자산', '배출권', '기타비유동자산', '기타금융업자산',
                       '부채', '유동부채', '단기사채', '단기차입금', '유동성장기부채', '유동금융부채',
                       '매입채무및기타유동채무', '유동종업원급여충당부채', '기타단기충당부채', '당기법인세부채',
                       '계약부채', '반품환불부채', '배출부채', '기타유동부채',
                       '매각예정으로분류된처분자산집단에포함된부채', '비유동부채', '사채', '장기차입금',
                       '비유동금융부채', '장기매입채무및기타비유동채무', '비유동종업원급여충당부채', '기타장기충당부채',
                       '이연법인세부채', '장기당기법인세부채', '계약부채', '반품환불부채', '배출부채',
                       '기타비유동부채', '기타금융업부채', '자본', '지배기업주주지분', '자본금', '신종자본증권',
                       '자본잉여금', '기타자본', '기타포괄손익누계액', '이익잉여금결손금', '비지배주주지분']
        example     :
                     자산  유동자산  비유동자산  기타금융업자산  부채  유동부채  비유동부채  ...  이익잉여금결손금
            2020/12  3615      2577        1038             NaN   242       219          23  ...              3265
            2021/12  4664      3357        1306             NaN   487       460          27  ...              4068
            2022/12  5315      3770        1545             NaN   383       364          19  ...              4823
            2023/3Q  5773      4194        1579             NaN   460       438          22  ...              5205

    @foreignExhaustRate
        constraint  : property
        type        : DataFrame (Multi-Indexed)
        description : time-series of foreign hold(exhuast) rate
        columns     : MultiIndex([('3M', '종가'),
                                  ('3M', '비중'),
                                  ('1Y', '종가'),
                                  ('1Y', '비중'),
                                  ('3Y', '종가'),
                                  ('3Y', '비중')
                                ])
        example     :
                                    3M              1Y              3Y
                           종가   비중     종가   비중     종가   비중
            날짜
            2020-11-01      NaN    NaN      NaN    NaN  10105.0  25.67
            2020-12-01      NaN    NaN      NaN    NaN  10033.0  25.21
            2021-01-01      NaN    NaN      NaN    NaN   9629.0  25.05
            ...             ...    ...      ...    ...      ...    ...
            2023-11-20  12490.0  37.18      NaN    NaN      NaN    NaN
            2023-11-21  12720.0  37.34      NaN    NaN      NaN    NaN
            2023-11-22  12700.0  37.36  12604.0  37.25      NaN    NaN

    @growthRate
        constraint  : property
        type        : DataFrame (multi frames included)
        description : growth rate (YoY)
        example     :
            [Y]
                     매출액증가율  판매비와관리비증가율  영업이익증가율  EBITDA증가율  EPS증가율
            2019/12          13.3                  -4.2            11.5           9.8        8.5
            2020/12          18.2                   8.8            21.4          21.4        4.9
            2021/12          39.2                  29.7            50.4          47.0       87.5
            2022/12          15.1                  27.3            16.7          16.4       10.2
            2023/3Q         -27.0                 -24.8           -30.4         -28.0      -21.4

            [Q]
                     매출액증가율  영업이익증가율  EBITDA증가율  EPS증가율
            2022/09           3.8           -31.4         -16.0      -24.2
            2022/12          -8.0           -69.0         -40.5      120.8
            2023/03         -18.1           -95.5         -57.2      -87.4
            2023/06         -22.3           -95.3         -57.3      -85.9
            2023/3Q         -12.2           -77.6         -41.4      -39.8

    @incomeStatement
        constraint  : property
        type        : DataFrame (multi frames included)
        description : income statement
        columns     : ['매출액', '매출원가', '매출총이익', '판매비와관리비', '영업이익', '영업이익발표기준',
                       '금융수익', '금융원가', '기타수익', '기타비용', '종속기업,공동지배기업및관계기업관련손익',
                       '세전계속사업이익', '법인세비용', '계속영업이익', '중단영업이익', '당기순이익',
                       '지배주주순이익', '비지배주주순이익']
        example     :
                     매출액 매출원가 매출총이익 판매비와관리비 영업이익 금융수익 금융원가 기타수익 기타비용 ... 당기순이익
            2020/12  319004   210898     108106          57980    50126    33279    19804      848     1716 ...      47589
            2021/12  429978   240456     189522   		 65419   124103    23775    14699     1161     1804 ...      96162
            2022/12  446216   289937     156279  		 88184    68094    37143    50916     2414    18019 ...      22417
            2023/2Q  123940   152172     -28231  		 34613   -62844    15203    25144      261      739 ...     -55734

    @marketShares
        constraint  : property
        type        : DataFrame
        description : market shares of the products, mostly not provided
        example     :
                    IC TEST SOCKET 류   LEENO PIN 류          상품     상품 등  의료기기 부품류          합계
                        내수     수출    내수   수출    내수  수출  내수  수출       내수  수출   내수   수출
            2020/12      NaN      NaN    NaN    NaN     NaN   NaN    NaN   NaN       NaN    NaN    NaN    NaN
            2021/12      NaN      NaN    NaN    NaN     NaN   NaN    NaN   NaN       NaN    NaN    NaN    NaN
            2022/12    10.50    89.50  27.80  72.20  100.00  0.00  95.80  4.20     99.30   0.70  24.30  75.70

    @multipleBands
        constraint  : property
        type        : DataFrame
        description : multiple band provided by fnguide
        example     :
                                                PER  ...                           PBR
                           종가     2.46X     3.42X  ...     0.38X     0.46X     0.54X
            날짜                                     ...
            2018-12-01      NaN       NaN       NaN  ...       NaN       NaN       NaN
            2019-01-01      NaN       NaN       NaN  ...       NaN       NaN       NaN
            2019-02-01  14800.0       NaN       NaN  ...       NaN       NaN       NaN
            ...             ...       ...       ...  ...       ...       ...       ...
            2025-10-01      NaN  10455.54  14535.75  ...  17745.94  21650.05  25554.16
            2025-11-01      NaN  10488.84  14582.04  ...  17844.72  21770.56  25696.39
            2025-12-01      NaN  10522.13  14628.33  ...  17943.50  21891.06  25838.63

    @multiplesOutstanding
        constraint  : property
        type        : Series
        description : multiples outstanding
        example     :
            [stock]
            fiscalPE         2.90
            forwardPE        3.06
            sectorPE         6.17
            priceToBook      0.32
            dividendYield    9.03
            dtype: float64

            [ETF]
            dividendYield     1.68
            fiscalPE         12.58
            priceToBook       1.15
            dtype: float64

    @products
        constraint  : property
        type        : DataFrame
        description : products of the corp
        example     :
                    유가증권평가및처분이익  이자수익  수수료수익  외환거래이익  기타(계)
            기말
            2019/12                  41.49     46.58        7.53          2.65      1.75
            2020/12                  56.93     33.26        5.92          2.65      1.24
            2021/12                  50.70     36.39        7.19          2.07      3.65
            2022/12                  54.21     34.58        5.27          3.31      2.63

    @profitRate
        constraint  : property
        type        : DataFrame
        description : profit rate
        example     :
                     매출총이익율  세전계속사업이익률  영업이익률  EBITDA마진율   ROA   ROE  ROIC
            2019/12          43.5                41.7        37.7          42.6  17.4  18.8  44.0
            2020/12          44.1                36.5        38.7          43.7  16.1  17.4  48.6
            2021/12          46.8                49.6        41.8          46.2  25.1  27.5  69.0
            2022/12          48.0                47.8        42.4          46.7  22.9  25.1  70.6
            2023/3Q          48.3                54.7        42.6          47.9  20.1  21.7  56.0

    @shareHolders
        constraint  : property
        type        : Series
        description : stock shares holded by affiliate person
        example     :
            최대주주등    9.13
            5%이상주주    12.02
            임원          0.04
            자기주식      0.66
            공시제외주주  78.15
            dtype: float64

    @shareInstitutes
        constraint  : property
        type        : DataFrame
        description : stock shares holded by institutes

    @shortBalance
        constraint  : property
        type        : DataFrame
        description : short balance
        example     :
                       대차잔고비중   종가
            날짜
            2022-10-17        3.50  139900
            2022-10-24        3.50  140000
            2022-10-31        3.44  136800
            ...                ...     ...
            2023-10-02        9.45  153800
            2023-10-09        9.52  154700
            2023-10-16        8.69  156800

    @shortSell
        constraint  : property
        type        : DataFrame
        description : short sell ratio
        example     :
                        공매도비중     종가
            날짜
            2022-11-28        1.37  12150.0
            2022-12-05        5.70  12800.0
            2022-12-12        1.75  12850.0
            ...                ...      ...
            2023-11-06        8.32  12570.0
            2023-11-13        0.09  12400.0
            2023-11-20        0.13  12490.0

    @snapShot
        constraint  : property
        type        : Series
        description : snap shot of the asset
        example     :
            date                 2023/11/17
            previousClose             12510
            fiftyTwoWeekHigh          13480
            fiftyTwoWeekLow           10950
            marketCap                 94069
            sharesOutstanding     751949461
            floatShares           663064556
            volume                   868029
            foreignRate                37.2
            beta                    0.74993
            return1M                    0.0
            return3M                  10.12
            return6M                   6.83
            return1Y                   5.13
            return3Y                  26.36
            dtype: object

    @stabilityRate
        constraint  : property
        type        : DataFrame
        description : stability rate of the corp.
        example     :
                     유동비율  당좌비율  부채비율  유보율  순차입금비율  이자보상배율  자기자본비율
            2019/12     980.4     932.0       8.5  3869.6           NaN        8094.6          92.2
            2020/12    1175.7    1119.3       7.2  4357.2           NaN       12174.7          93.3
            2021/12     730.0     704.8      11.7  5411.7           NaN       17775.4          89.6
            2022/12    1036.3    1000.2       7.8  6402.1           NaN       18929.7          92.8
            2023/3Q     956.9     924.2       8.7  6902.7           NaN       11841.9          92.0
    """

    def __init__(self, ticker:str):
        self.url = _url(ticker)
        return

    @property
    def abstract(self) -> DataFrame:
        def _get_(index:int) -> DataFrame:
            data = web.list(self.url.snapshot)[index]
            data = data.set_index(keys=[data.columns[0]])
            if isinstance(data.columns[0], tuple):
                data.columns = data.columns.droplevel()
            else:
                data.columns = data.iloc[0]
                data = data.drop(index=data.index[0])
            data = data.T
            data = data.head(len(data) - len([i for i in data.index if i.endswith(')')]) + 1)
            data.index.name = '기말'
            data.index = [
                idx.replace("(E) : Estimate 컨센서스, 추정치 ", "").replace("(P) : Provisional 잠정실적 ", "")
                for idx in data.index
            ]
            data.columns.name = None
            for col in data:
                data[col] = data[col].apply(str2num)
            data = data.drop(columns=[col for col in data.columns if "발표기준" in col])
            data = data.rename(columns={col:col[:col.find("(")] if "(" in col else col for col in data.columns})
            if index in [12, 15]:
                data.index = [
                    col.replace("03","1Q").replace("06","2Q").replace("09","3Q").replace("12","4Q") for col in data.index
                ]
            return data
        return multiframes(dict(
            Y=_get_(11 if self.url.gb == 'D' else 14),
            Q=_get_(12 if self.url.gb == 'D' else 15)
        ))

    @property
    def benchmarkMultiples(self) -> DataFrame:
        json = web.json(self.url.cdn.benchmarkMultiples)
        def _get_(key: str) -> DataFrame:
            head = DataFrame(json[f'{key}_H'])[['ID', 'NAME']].set_index(keys='ID')
            head['NAME'] = head['NAME'].str.replace("'", "20")
            head = head.to_dict()['NAME']
            head.update({'CD_NM': '이름'})
            data = DataFrame(json[key])[head.keys()].rename(columns=head).set_index(keys='이름')
            data.index.name = None
            return data.replace('-', nan).T.astype(float)
        return concat(
            objs={'PER': _get_('02'), 'EV/EBITDA': _get_('03'), 'ROE': _get_('04'), '배당수익률': _get_('05')},
            axis=1
        )

    @property
    def businessSummary(self) -> str:
        html = web.html(self.url.snapshot).find('ul', id='bizSummaryContent').find_all('li')
        t = '\n\n '.join([e.text for e in html])
        w = [
            '.\n' if t[n] == '.' and not any([t[n - 1].isdigit(), t[n + 1].isdigit(), t[n + 1].isalpha()]) else t[n]
            for n in range(1, len(t) - 2)
        ]
        s = f' {t[0]}{str().join(w)}{t[-2]}{t[-1]}'
        return s.replace(' ', '').replace('\xa0\xa0', ' ').replace('\xa0', ' ').replace('\n ', '\n')

    @property
    def cashFlow(self) -> DataFrame:
        cut = ['계산에 참여한 계정 펼치기', '(', ')', '*', '&nbsp;', ' ', " "]
        col = {
            "영업활동으로인한현금흐름": "영업현금흐름",
            "투자활동으로인한현금흐름": "투자현금흐름",
            "재무활동으로인한현금흐름": "재무현금흐름",
            "환율변동효과": "환율변동손익",
            "기말현금및현금성자산": "현금및현금성자산"
        }
        def _get_(index:int) -> DataFrame:
            data = web.list(self.url.finance)[index]
            data = data.set_index(keys=[data.columns[0]])
            data = data.drop(columns=[c for c in data if not c.startswith('20')])
            data.index.name = None
            data.columns = data.columns.tolist()[:-1] + [f"{data.columns[-1][:4]}/{int(data.columns[-1][-2:]) // 3}Q"]
            data.index = [cutString(x, cut) for x in data.index]
            data = data.T
            if index == 5:
                data.index = [
                    c.replace("03", "1Q").replace("06", "2Q").replace("09", "3Q").replace("12", "4Q") for c in data.index
                ]
            return data.rename(columns=col).fillna(0).astype(int)
        return multiframes(dict(
            Y=_get_(4),
            Q=_get_(5)
        ))

    @property
    def consensusOutstanding(self) -> Series:
        src = web.list(self.url.snapshot)[7]
        data = []
        for dat in src.iloc[0].tolist():
            try:
                data.append(float(dat))
            except ValueError:
                data.append(nan)
        return Series(dict(zip(src.columns.tolist(), data)))

    @property
    def consensusPrice(self) -> DataFrame:
        cols = {'TRD_DT': '날짜', 'VAL1': '투자의견', 'VAL2': '컨센서스', 'VAL3': '종가'}
        data = web.data(self.url.cdn.priceConsensus, "CHART")
        data = data.rename(columns=cols).set_index(keys='날짜')
        data.index = to_datetime(data.index)
        for col in data:
            data[col] = data[col].apply(str2num)
        data['격차'] = round(100 * (data['종가'] / data['컨센서스'] - 1), 2)
        return data.astype(float)

    @property
    def consensusProfit(self) -> DataFrame:
        cols = {
            "GS_YM": "기말",
            "SALES_R": "매출실적", "SALES_F": "매출전망",
            "OP_R": "영업이익실적", "OP_F": "영업이익전망"
        }
        yy = web.data(self.url.cdn.profitConsensusAnnual, "CHART")[cols.keys()].rename(columns=cols).set_index(keys="기말")
        qq = web.data(self.url.cdn.profitConsensusQuarter, "CHART")[cols.keys()].rename(columns=cols).set_index(keys="기말")
        for y, q in zip(yy, qq):
            yy[y] = yy[y].apply(str2num)
            qq[q] = qq[q].apply(str2num)
        return multiframes(dict(Y=yy, Q=qq))

    @property
    def consensusThisFiscalYear(self) -> DataFrame:
        cols = {
            "STD_DT": "날짜",
            "SALES": "매출", "SALES_MAX": "매출(최대)", "SALES_MIN": "매출(최소)",
            "OP": "영업이익", "OP_MAX": "영업이익(최대)", "OP_MIN": "영업이익(최소)",
            "EPS": "EPS", "EPS_MAX": "EPS(최대)", "EPS_MIN": "EPS(최소)",
            "PER": "PER", "PER_MAX": "PER(최대)", "PER_MIN": "PER(최소)", "PER_12F": "12M PER"
        }
        data = web.data(self.url.cdn.abstractConsensusRelevant, "CHART")
        if data.empty:
            return DataFrame(columns=list(cols.values()))
        data = data[cols.keys()].rename(columns=cols).set_index(keys='날짜')
        for col in data:
            data[col] = data[col].apply(str2num)
        return data

    @property
    def consensusNextFiscalYear(self) -> DataFrame:
        cols = {
            "STD_DT": "날짜",
            "SALES": "매출", "SALES_MAX": "매출(최대)", "SALES_MIN": "매출(최소)",
            "OP": "영업이익", "OP_MAX": "영업이익(최대)", "OP_MIN": "영업이익(최소)",
            "EPS": "EPS", "EPS_MAX": "EPS(최대)", "EPS_MIN": "EPS(최소)",
            "PER": "PER", "PER_MAX": "PER(최대)", "PER_MIN": "PER(최소)", "PER_12F": "12M PER"
        }
        data = web.data(self.url.cdn.abstractConsensusForward, "CHART")
        if data.empty:
            return DataFrame(columns=list(cols.values()))
        data = data[cols.keys()].rename(columns=cols).set_index(keys='날짜')
        for col in data:
            data[col] = data[col].apply(str2num)
        return data

    @property
    def expenses(self) -> DataFrame:
        json = web.json(self.url.cdn.expenses)
        def _get_(period: str) -> DataFrame:
            manage = DataFrame(json[f"05_{period}"]).set_index(keys="GS_YM")["VAL1"]
            cost = DataFrame(json[f"06_{period}"]).set_index(keys="GS_YM")["VAL1"]
            manage.index.name = cost.index.name = '기말'
            data = concat({"판관비율": manage, "매출원가율": cost}, axis=1)
            for col in data:
                data[col] = data[col].apply(str2num)
            if period == "Q":
                data.index = [
                    c.replace("03", "1Q").replace("06", "2Q").replace("09", "3Q").replace("12", "4Q") for c in data.index
                ]
            return data
        return multiframes(dict(Y=_get_('Y'), Q=_get_('Q')))

    @property
    def financialStatement(self) -> DataFrame:
        cutter = ['계산에 참여한 계정 펼치기', '(', ')', '*', '&nbsp;', ' ', " "]
        def _get_(period:str) -> DataFrame:
            data = web.list(self.url.finance)[{"Y": 2, "Q": 3}[period]]
            data = data.set_index(keys=[data.columns[0]])
            data = data.drop(columns=[col for col in data if not col.startswith('20')])
            data.index.name = None
            data.columns = data.columns.tolist()[:-1] + [f"{data.columns[-1][:4]}/{int(data.columns[-1][-2:]) // 3}Q"]
            data.index = [cutString(x, cutter) for x in data.index]
            data = data.T.astype(float)
            if period == "Q":
                data.index = [
                    c.replace("03", "1Q").replace("06", "2Q").replace("09", "3Q").replace("12", "4Q") for c in data.index
                ]
            return data
        return multiframes(dict(Y=_get_("Y"), Q=_get_("Q")))

    @property
    def foreignExhaustRate(self) -> DataFrame:
        urls = [self.url.cdn.foreignRate3Months, self.url.cdn.foreignRate1Year, self.url.cdn.foreignRate3Years]
        cols = {'TRD_DT': '날짜', 'J_PRC': '종가', 'FRG_RT': '비중'}
        objs = {}
        for _url_ in urls:
            data = web.data(_url_, "CHART")[cols.keys()]
            data = data.rename(columns=cols).set_index(keys='날짜')
            data.index = to_datetime(data.index)
            for col in data:
                data[col] = data[col].apply(str2num)
            objs[_url_[_url_.rfind('_') + 1: _url_.rfind('.')]] = data
        return concat(objs=objs, axis=1)

    @property
    def growthRate(self) -> DataFrame:
        cutter = ['계산에 참여한 계정 펼치기', '(', ')', '*', '&nbsp;', ' ', " "]
        def _get_(index:int):
            data = web.list(self.url.ratio, displayed_only=True)[index]
            cols = data[data.columns[0]].tolist()
            data = data.iloc[cols.index('성장성비율') + 1: cols.index('수익성비율')]
            data = data.set_index(keys=[data.columns[0]])
            data = data.drop(columns=[col for col in data if not col.startswith('20')])
            data.index.name = None
            data.columns = data.columns.tolist()[:-1] + [f"{data.columns[-1][:4]}/{int(data.columns[-1][-2:]) // 3}Q"]
            data.index = [cutString(x, cutter) for x in data.index]
            data = data.T.astype(float)
            if index == 1:
                data.index = [
                    c.replace("03", "1Q").replace("06", "2Q").replace("09", "3Q").replace("12", "4Q") for c in data.index
                ]
            return data
        return multiframes(dict(Y=_get_(0), Q=_get_(1)))

    @property
    def incomeStatement(self) -> DataFrame:
        cutter = ['계산에 참여한 계정 펼치기', '(', ')', '*']
        def _get_(period:str) -> DataFrame:
            data = web.list(self.url.finance)[{"Y": 0, "Q": 1}[period]]
            data = data.set_index(keys=[data.columns[0]])
            data = data.drop(columns=[col for col in data if not col.startswith('20')])
            data.index.name = None
            data.columns = data.columns.tolist()[:-1] + [f"{data.columns[-1][:4]}/{int(data.columns[-1][-2:]) // 3}Q"]
            data.index = [cutString(x, cutter) for x in data.index]
            data = data.T.astype(float)
            if period == "Q":
                data.index = [
                    c.replace("03", "1Q").replace("06", "2Q").replace("09", "3Q").replace("12", "4Q") for c in data.index
                ]
            return data
        return multiframes(dict(Y=_get_("Y"), Q=_get_("Q")))

    @property
    def marketShares(self) -> DataFrame:
        src = web.list(self.url.corp)[{'D': 10, 'B': 11}[self.url.gb]]
        data = src[src.columns[1:]].set_index(keys=src.columns[1])
        data = data.T.copy()
        if all([i.startswith("Unnamed") for i in data.index]):
            return DataFrame(columns=["내수", "수출"])
        data.columns = [col.replace("\xa0", " ") for col in data.columns]

        domestic = data[data[data.columns[0]] == "내수"].drop(columns=data.columns[0])
        exported = data[data[data.columns[0]] == "수출"].drop(columns=data.columns[0])
        domestic.index = exported.index = [i.replace('.1', '') for i in domestic.index]
        domestic.columns.name = exported.columns.name = None
        data = concat(objs={"내수": domestic, "수출": exported}, axis=1)
        # return data  # 내수/수출 구분 우선 시
        data = concat(objs={(c[1], c[0]): data[c] for c in data}, axis=1)
        return data[sorted(data.columns, key=lambda x: x[0])]  # 상품 구분 우선 시

    @property
    def multipleBand(self) -> DataFrame:
        json = web.json(self.url.cdn.multipleBands)
        def _get_(key: str) -> DataFrame:
            head = DataFrame(json[key])[['ID', 'NAME']].set_index(keys='ID')
            head = head.to_dict()['NAME']
            head.update({'GS_YM': '날짜', 'PRICE': '종가'})
            data = DataFrame(json['CHART']).rename(columns=head)[head.values()]
            data["날짜"] = to_datetime(data["날짜"])
            data = data.set_index(keys='날짜')
            if "0.00X" in data:
                # data = data.T.drop_duplicates().T
                data = data.drop(columns=['0.00X'])
            data = data.dropna(how='all', axis=1)
            for col in data:
                data[col] = data[col].apply(str2num)
            return data
        return concat(objs={'PER': _get_('CHART_E'), 'PBR': _get_('CHART_B')}, axis=1)

    @property
    def multiplesOutstanding(self) -> Series:
        src = web.html(self.url.snapshot).find('div', id='corp_group2')
        src = [val for val in src.text.split('\n') if val]
        data = {
            "fiscalPE": str2num(src[src.index('PER') + 1]),
            "forwardPE": str2num(src[src.index('12M PER') + 1]),
            "sectorPE": str2num(src[src.index('업종 PER') + 1]),
            "priceToBook": str2num(src[src.index('PBR') + 1]),
            "dividendYield": str2num(src[src.index('배당수익률') + 1]),
        }
        return Series(data)

    @property
    def products(self) -> DataFrame:
        json = web.json(self.url.cdn.products)
        head = DataFrame(json['chart_H'])[['ID', 'NAME']].set_index(keys='ID').to_dict()['NAME']
        head.update({'PRODUCT_DATE': '기말'})
        values = list(head.values())
        for n, (key, value) in enumerate(head.items()):
            if value in values[:n]:
                head[key] = f'{value}_Copy'
        data = DataFrame(json['chart']).rename(columns=head).set_index(keys='기말')
        data = data.drop(columns=[c for c in data.columns if data[c].astype(float).sum() == 0])

        i = data.columns[-1]
        data['Sum'] = data.astype(float).sum(axis=1)
        data = data[(90 <= data.Sum) & (data.Sum < 110)].astype(float)
        data[i] = data[i] - (data.Sum - 100)
        return data.drop(columns=['Sum'])

    @property
    def profitRate(self) -> DataFrame:
        cutter = ['계산에 참여한 계정 펼치기', '(', ')', '*', '&nbsp;', ' ', " "]
        data = web.list(self.url.ratio)[0]
        cols = data[data.columns[0]].tolist()
        idet = cols.index('수익성비율') + 1
        iend = cols.index('활동성비율') if "활동성비율" in cols else len(cols) - 1
        data = data.iloc[idet: iend]
        data = data.set_index(keys=[data.columns[0]])
        data = data.drop(columns=[col for col in data if not col.startswith('20')])
        data.index.name = None
        data.columns = data.columns.tolist()[:-1] + [f"{data.columns[-1][:4]}/{int(data.columns[-1][-2:]) // 3}Q"]
        data.index = [cutString(x, cutter) for x in data.index]
        return data.T.astype(float)

    @property
    def shareHolders(self) -> Series:
        data = web.data(self.url.cdn.shares).replace("", nan)
        return Series(index=data["NM"].values, data=data["STK_RT"].values, dtype=float).dropna()

    @property
    def shareInstitutes(self) -> Series:
        data = web.list(self.url.snapshot)[2]
        return data.replace("관련 데이터가 없습니다.", nan)

    @property
    def shortBalance(self) -> DataFrame:
        cols = {'TRD_DT': '날짜', 'BALANCE_RT': '대차잔고비중', 'ADJ_PRC': '종가'}
        data = web.data(self.url.cdn.shortBalance, "CHART").rename(columns=cols)[cols.values()].set_index(keys='날짜')
        data.index = to_datetime(data.index)
        return data.replace("", nan).astype(float)

    @property
    def shortSell(self) -> DataFrame:
        cols = {'TRD_DT': '날짜', 'VAL': '공매도비중', 'ADJ_PRC': '종가'}
        data = web.data(self.url.cdn.shortSell, "CHART").rename(columns=cols).set_index(keys='날짜')
        data.index = to_datetime(data.index)
        return data.replace("", nan).astype(float)

    @property
    def snapShot(self) -> Series:
        src = web.html(self.url.xml).find('price')
        return Series({
            "date": src.find("date").text,
            "previousClose": str2num(src.find("close_val").text),
            "fiftyTwoWeekHigh": str2num(src.find("high52week").text),
            "fiftyTwoWeekLow": str2num(src.find("low52week").text),
            "marketCap": str2num(src.find("mkt_cap_1").text),
            "sharesOutstanding": str2num(src.find("listed_stock_1").text),
            "floatShares": str2num(src.find("ff_sher").text),
            "volume": str2num(src.find("deal_cnt").text),
            "foreignRate": str2num(src.find("frgn_rate").text),
            "beta": str2num(src.find("beta").text),
            "return1M": str2num(src.find("change_1month").text),
            "return3M": str2num(src.find("change_3month").text),
            "return6M": str2num(src.find("change_6month").text),
            "return1Y": str2num(src.find("change_12month").text),
            "return3Y": str2num(src.find("change_36month").text),
        })

    @property
    def stabilityRate(self) -> DataFrame:
        cutter = ['계산에 참여한 계정 펼치기', '(', ')', '*', '&nbsp;', ' ', " "]
        data = web.list(self.url.ratio)[0]
        cols = data[data.columns[0]].tolist()
        data = data.iloc[cols.index('안정성비율') + 1: cols.index('성장성비율')]
        data = data.set_index(keys=[data.columns[0]])
        data = data.drop(columns=[col for col in data if not col.startswith('20')])
        data.index.name = None
        data.columns = data.columns.tolist()[:-1] + [f"{data.columns[-1][:4]}/{int(data.columns[-1][-2:]) // 3}Q"]
        data.index = [cutString(x, cutter) for x in data.index]
        return data.T.astype(float)


if __name__ == "__main__":
#     from pandas import set_option
#     set_option('display.expand_frame_repr', False)

    fn = fnguide(
        # "005930"
        # "069500"
        "036890"
    )
#     print(fn.abstract)
#     print(fn.abstract.Y)
#     print(fn.abstract.Q)
#     print(fn.benchmarkMultiples)
#     print(fn.businessSummary)
#     print(fn.cashFlow)
    # print(fn.consensusOutstanding)
#     print(fn.consensusPrice)
    # print(fn.consensusProfit)
    # print(fn.consensusProfit.Y)
    # print(fn.consensusProfit.Q)
#     print(fn.consensusThisFiscalYear)
    # print(fn.consensusNextFiscalYear)
#     print(fn.expenses)
    # print(fn.expenses.Y)
#     print(fn.expenses.Q)
#     print(fn.financialStatement)
    # print(fn.financialStatement.Y)
#     print(fn.financialStatement.Q)
#     print(fn.foreignExhaustRate)
#     print(fn.growthRate)
    # print(fn.growthRate.Y)
#     print(fn.growthRate.Q)
#     print(fn.incomeStatement)
    # print(fn.incomeStatement.Y)
#     print(fn.incomeStatement.Q)
    # print(fn.marketShares)
#     print(fn.multipleBand)
#     print(fn.multiplesOutstanding)
    print(fn.products)
    # print(fn.profitRate)
    # print(fn.shareHolders)
    # print(fn.shareInstitutes)
#     print(fn.shortBalance)
#     print(fn.shortSell)
#     print(fn.snapShot)
    # print(fn.stabilityRate)



         건설중장비부품  연료전지부품  건설중장비부품_Copy  연료전지부품_Copy
기말                                                 
2021/12     0.00    0.00         96.48         3.52
2022/12    96.86    3.14          0.00         0.00
2023/12    97.40    2.60          0.00         0.00
2024/12    96.39    3.61          0.00         0.00


#### wrapper.py

In [ ]:
# try:
#     from .krx import PyKrx
#     from .fnguide import fnguide
# except ImportError:
#     from src.fetch.stock.krx import PyKrx
#     from src.fetch.stock.fnguide import fnguide
from pandas import concat
from time import perf_counter
from typing import List


class CacheStock:

    _log: List[str] = []
    def __init__(self, *tickers:str):
        self.log = f'  >> RUN [CACHING STOCK DATA]: '
        stime = perf_counter()

        ohlcv = {}
        marketCap = {}
        perBand = {}
        foreignRate = {}
        for ticker in tickers:
            krx = PyKrx(ticker)
            fng = fnguide(ticker)

            try:
                ohlcv[ticker] = krx.ohlcv
            except Exception as reason:
                self.log = f'     ...FAILED TO FETCH OHLCV: {ticker} / {reason}'

            try:
                marketCap[ticker] = krx.getMarketCap()["시가총액"]
            except Exception as reason:
                self.log = f'     ...FAILED TO FETCH MARKET CAP: {ticker} / {reason}'

            try:
                perBand[ticker] = fng.multipleBand["PER"]
            except Exception as reason:
                self.log = f'     ...FAILED TO FETCH PER BAND: {ticker} / {reason}'

            try:
                foreignRate[ticker] = fng.foreignExhaustRate
            except Exception as reason:
                self.log = f'     ...FAILED TO FETCH FOREIGN EXHAUST RATE: {ticker} / {reason}'

        self.ohlcv = concat(ohlcv, axis=1)
        self.marketCap = concat(marketCap, axis=1)
        self.perBand = concat(perBand, axis=1)
        self.foreignRate = concat(foreignRate, axis=1)

        self._log[0] += f'{len(tickers):,d} items @{self.ohlcv.index.astype(str).values[-1]}'.replace("-", "/")
        self.log = f'  >> END: {perf_counter() - stime:.2f}s'
        return

    @property
    def log(self) -> str:
        return "\n".join(self._log)

    @log.setter
    def log(self, log: str):
        self._log.append(log)

if __name__ == "__main__":
    cache = CacheStock("005930", "000660")
    print(cache.log)
    print(cache.ohlcv)
    # print(cache.marketCap)
    # print(cache.perBand)
    # print(cache.foreignRate)

  >> RUN [CACHING STOCK DATA]: 2 items @2025/07/11
  >> END: 9.03s
           005930                                 000660                                  
             open   high    low  close    volume    open    high     low   close    volume
date                                                                                      
2015-07-14  25300  25400  24420  24500    399794   40600   40650   37800   37850  15180511
2015-07-15  24500  24760  24480  24700    167413   38400   39400   38350   39300   8281669
2015-07-16  24460  25740  24460  25640    223469   39000   39500   38650   39450   2373825
2015-07-17  26000  26220  25560  26100    297233   39500   40500   38700   39950   3352178
2015-07-20  25820  26080  25460  25500    128928   40150   40350   38650   38650   2535208
...           ...    ...    ...    ...       ...     ...     ...     ...     ...       ...
2025-07-07  62900  63300  61700  61700  17164708  266000  272000  264500  271000   2523564
2025-07-08  61600  6240

In [ ]:
cache.foreignRate


005930                                           000660                                        
                 3M              1Y              3Y               3M               1Y              3Y       
                 종가     비중       종가     비중       종가     비중        종가     비중        종가     비중       종가     비중
날짜                                                                                                          
2022-07-01      NaN    NaN      NaN    NaN  60447.0  49.75       NaN    NaN       NaN    NaN  98467.0  49.71
2022-08-01      NaN    NaN      NaN    NaN  60273.0  49.81       NaN    NaN       NaN    NaN  95223.0  49.99
2022-09-01      NaN    NaN      NaN    NaN  55595.0  49.48       NaN    NaN       NaN    NaN  88195.0  50.12
2022-10-01      NaN    NaN      NaN    NaN  56711.0  49.57       NaN    NaN       NaN    NaN  91137.0  50.73
2022-11-01      NaN    NaN      NaN    NaN  61091.0  49.90       NaN    NaN       NaN    NaN  86918.0  50.83
...             ...    ...      ...    ...      ...    ...       ...    ...       ...    ...      ...    ...
2025-07-04  63300.0  49.79      NaN    NaN      NaN    NaN  270500.0  55.46       NaN    NaN      NaN    NaN
2025-07-07  61700.0  49.75      NaN    NaN      NaN    NaN  271000.0  55.42       NaN    NaN      NaN    NaN
2025-07-08  61400.0  49.72      NaN    NaN      NaN    NaN  282000.0  55.44       NaN    NaN      NaN    NaN
2025-07-09  60400.0  49.66      NaN    NaN      NaN    NaN  281000.0  55.43       NaN    NaN      NaN    NaN
2025-07-10  61000.0  49.68  61560.0  49.72      NaN    NaN  297000.0  55.47  280300.0  55.44      NaN    NaN

[135 rows x 12 columns]

## build

### apps

#### stock.py

In [6]:
# try:
#     from ...common.env import FILE, dDict
#     from ...common.util import krw2currency, str2num
#     from ...fetch.stock.fnguide import fnguide
# except ImportError:
#     from src.common.env import FILE, dDict
#     from src.common.util import krw2currency, str2num
#     from src.fetch.stock.fnguide import fnguide
from datetime import timedelta
from json import dumps
from pandas import DataFrame, Series, DateOffset
from pandas import concat, read_parquet, to_datetime, isna
from scipy.stats import linregress
from ta.trend import MACD
from ta.momentum import RSIIndicator
from time import perf_counter
from typing import List



class Stocks:

    _log: List[str] = []
    def __init__(self):
        stime = perf_counter()
        self.log = f'  >> BUILD [STOCK]: '
        self.basis = basis = read_parquet(FILE.BASELINE, engine='pyarrow')
        self.price = price = read_parquet(FILE.PRICE, engine='pyarrow')
        self.mcap = mcap = read_parquet(FILE.MARKET_CAP, engine='pyarrow')
        self.band = band = read_parquet(FILE.PER_BAND, engine='pyarrow')
        self.foreignRage = foreignRate = read_parquet(FILE.FOREIGN_RATE, engine='pyarrow')
        self.astat = astat = read_parquet(FILE.ANNUAL_STATEMENT, engine='pyarrow')
        self.qstat = qstat = read_parquet(FILE.QUARTER_STATEMENT, engine='pyarrow')

        tickers = price.columns.get_level_values(0).unique()
        xrange = [
            price[price.index >= (price.index[-1] - DateOffset(months=6))].index[0],
            to_datetime(price.index[-1])
        ]

        __mem__ = dDict()
        for ticker in tickers:
            if not ticker in basis.index:
                self.log = f'     ...TICKER NOT FOUND IN BASELINE: {ticker}'
                continue
            fng = fnguide(ticker)
            try:
                product = self.convertProduct(fng.products)
            except Exception as reason:
                product = 'null'
                self.log = f'     ...PRODUCT FOR: {ticker} NOT FOUND: {reason}'
            general = basis.loc[ticker]
            ohlcv = price[ticker].dropna().astype(int)
            typical = (ohlcv.close + ohlcv.high + ohlcv.low) / 3
            monthlyClose = ohlcv["close"].resample('M').nearest()
            monthlyClose.index = monthlyClose.index.strftime("%Y/%m")
            annualClose = monthlyClose[
                monthlyClose.index.str.endswith('/12') | \
                (monthlyClose.index == monthlyClose.index[-1])
            ].rename(index={monthlyClose.index[-1]:f'{monthlyClose.index[-1][:4]}/현재'})

            trend = self.calcTrend(typical)
            annual = astat[ticker].map(str2num)
            quarter = qstat[ticker].map(str2num)
            cap = mcap[ticker]
            multipleBand = band[ticker] if ticker in band else DataFrame()
            foreignExhaustRate = foreignRate[ticker] if ticker in foreignRate else DataFrame()
            _xrange = xrange[0]
            if not _xrange in ohlcv.index:
                _xrange = ohlcv.index[0]
            try:
                __mem__[ticker] = dDict(
                    name=general['name'],
                    date=ohlcv.index.astype(str).tolist(),
                    spec=self.convertOverview(general),
                    xrange=[ohlcv.index.get_loc(_xrange), len(ohlcv) - 1],
                    ohlcv=self.convertOhlcv(ohlcv),
                    sma=self.convertSma(typical),
                    bollinger=self.convertBollinger(typical),
                    envelope=self.convertEnvelope(ohlcv, typical),
                    trend=self.convertTrend(trend),
                    macd=self.convertMacd(typical),
                    rsi=self.convertRsi(typical),
                    # deviation=self.convertDeviation(typical, trend),
                    sales_y=self.convertSales(annual, cap),
                    sales_q=self.convertSales(quarter, cap),
                    asset=self.convertAsset(annual, quarter),
                    growth=self.convertGrowth(annual),
                    per=self.convertPer(general),
                    pbr=self.convertPbr(annual, general),
                    div=self.convertDiv(annual, general),
                    peg = self.convertPeg(annual, general, annualClose),
                    perBand=self.convertPerBand(multipleBand, general),
                    foreignRate=self.convertForeignRate(foreignExhaustRate, general),
                    product=product
                )
            except Exception as reason:
                print(f'Error while collecting: {ticker}, {reason}')
                continue
        self.__mem__ = __mem__

        self._log[0] += f'{len(tickers):,d} items @{price.index.astype(str).values[-1]}'.replace("-", "/")
        self.log = f'  >> BUILD END: {perf_counter() - stime:.2f}s'
        return

    def __iter__(self):
        for ticker, attr in self.__mem__:
            yield ticker, attr

    @property
    def log(self) -> str:
        return "\n".join(self._log)

    @log.setter
    def log(self, log: str):
        self._log.append(log)

    @classmethod
    def calcTrend(cls, typical:Series) -> DataFrame:
        def _regression(subdata: Series, newName: str = '') -> Series:
            newName = newName if newName else subdata.name
            subdata.index.name = 'date'
            subdata = subdata.reset_index(level=0)
            xrange = (subdata['date'].diff()).dt.days.fillna(1).astype(int).cumsum()

            slope, intercept, _, _, _ = linregress(x=xrange, y=subdata[subdata.columns[-1]])
            fitted = slope * xrange + intercept
            fitted.name = newName
            return concat(objs=[subdata, fitted], axis=1).set_index(keys='date')[fitted.name]

        objs = [_regression(typical, '10년')]
        for yy in [5, 2, 1, 0.5, 0.25]:
            col = f"{yy}년" if isinstance(yy, int) else f"{int(yy * 12)}개월"
            date = typical.index[-1] - timedelta(int(yy * 365))
            if typical.index[0] > date:
                objs.append(Series(name=col, index=typical.index))
            else:
                objs.append(_regression(typical[typical.index >= date], col))
        return concat(objs, axis=1)

    @classmethod
    def convertOhlcv(cls, ohlcv:DataFrame) -> str:
        obj = {
            'open': ohlcv['open'],
            'high': ohlcv['high'],
            'low': ohlcv['low'],
            'close': ohlcv['close'],
            'volume': ohlcv['volume'],
        }
        for key in obj:
            obj[key] = obj[key].tolist()
        return dumps(obj).replace(" ", "").replace("NaN", "null")

    @classmethod
    def convertSma(cls, typical:Series) -> str:
        obj = {
            'sma5': typical.rolling(5).mean(),
            'sma20': typical.rolling(20).mean(),
            'sma60': typical.rolling(60).mean(),
            'sma120': typical.rolling(120).mean(),
            'sma200': typical.rolling(200).mean(),
        }
        for key in obj:
            obj[key] = round(obj[key], 1).tolist()
        return dumps(obj).replace(" ", "").replace("NaN", "null")

    @classmethod
    def convertBollinger(cls, typical:Series) -> str:
        obj = {
            'upper': typical.rolling(20).mean() + 2 * typical.rolling(20).std(),
            'upperTrend': typical.rolling(20).mean() + 1 * typical.rolling(20).std(),
            'middle': typical.rolling(20).mean(),
            'lower': typical.rolling(20).mean() - 2 * typical.rolling(20).std(),
            'lowerTrend': typical.rolling(20).mean() - 1 * typical.rolling(20).std(),
            # 'width': 100 * 4 * typical.rolling(20).std() / typical.rolling(20).mean()
        }
        for key in obj:
            obj[key] = round(obj[key], 1).tolist()
        return dumps(obj).replace(" ", "").replace("NaN", "null")

    @classmethod
    def convertEnvelope(cls, ohlcv:DataFrame, typical:Series) -> str:
        ma = typical.rolling(window=20).mean()
        adr = (ohlcv["high"] - ohlcv["low"]).rolling(window=14).mean()
        pct = 6 # [%]
        obj = {
            'adr': adr,
            # 'upper': ma + 1.5 * adr,
            # 'lower': ma - 1.5 * adr
            'upper': ma * (1 + pct / 100),
            'lower': ma * (1 - pct / 100)
        }
        for key in obj:
            obj[key] = round(obj[key], 1).tolist()
        return  dumps(obj).replace(" ", "").replace("NaN", "null")

    @classmethod
    def convertTrend(cls, trend:DataFrame) -> str:
        obj = {}
        for col in trend:
            series = trend[col].dropna()
            obj[col] = {
                'x': 'srcDate' if col == "10년" else  series.index.strftime("%Y-%m-%d").tolist(),
                'y': round(series, 1).tolist()
            }
        return dumps(obj).replace(" ", "").replace("NaN", "null").replace('"srcDate"', 'srcDate')

    @classmethod
    def convertMacd(cls, typical:Series):
        macd = MACD(close=typical)
        obj ={
            'macd': round(macd.macd(), 1).tolist(),
            'signal': round(macd.macd_signal(), 1).tolist(),
            'diff': round(macd.macd_diff(), 1).tolist()
        }
        return dumps(obj).replace(" ", "").replace("NaN", "null")

    @classmethod
    def convertRsi(cls, typical:Series):
        rsi = RSIIndicator(close=typical)
        obj = {
            'rsi': round(rsi.rsi(), 1).tolist()
        }
        return dumps(obj).replace(" ", "").replace("NaN", "null")

    @classmethod
    def convertSales(cls, statement:DataFrame, marketcap:DataFrame) -> str:
        sales = statement[statement.columns.tolist()[:3] + ['영업이익률(%)']].dropna(how='all')
        sales = sales.sort_index()
        for n, date in enumerate(sales.index):
            if "(" in date:
                sales = sales.iloc[:n + 1]
                if len(sales) > 5:
                    sales = sales.iloc[-5:]
        if not "(" in sales.index[-1]:
            if len(sales) > 4:
                sales = sales.iloc[-4:]

        columns = sales.columns
        if not marketcap.empty:
            marketcap.index = to_datetime(marketcap.index).strftime("%Y/%m")
            marketcap = marketcap[marketcap.index.isin(sales.index) | (marketcap.index == marketcap.index[-1])]
            if "(" in sales.index[-1]:
                marketcap = marketcap.rename(index={marketcap.index[-1]:sales.index[-1]})

            marketcap = Series(index=marketcap.index, data=marketcap['시가총액'] / 1e8, dtype=int)
            sales = concat([marketcap, sales], axis=1)

        e_sales = 1e8 * sales
        obj = {
            'index': sales.index.tolist(),
            'sales': sales[columns[0]].tolist(),
            'salesLabel': columns[0].replace("(억원)", ""),
            'salesText': [krw2currency(v) for v in e_sales[columns[0]]],
            'profit': sales[columns[1]].tolist(),
            'profitLabel': columns[1].replace("(억원)", ""),
            'profitText': [krw2currency(v) for v in e_sales[columns[1]]],
            'netProfit': sales[columns[2]].tolist(),
            'netProfitLabel': columns[2].replace("(억원)", ""),
            'netProfitText': [krw2currency(v) for v in e_sales[columns[2]]],
            'profitRate': sales['영업이익률(%)'].tolist(),
            'profitRateLabel': '영업이익률(%)'
        }
        if not marketcap.empty:
            obj['marketcap'] = sales['시가총액'].tolist()
            obj['marketcapLabel'] = '시가총액'
            obj['marketcapText'] = [krw2currency(v) for v in e_sales['시가총액']]
            obj['marketcapText'][-1] = f"(현재){obj['marketcapText'][-1]}"
        return dumps(obj).replace("NaN", "null")

    @classmethod
    def convertAsset(cls, a:DataFrame, q:DataFrame) -> str:
        cols = ['자산총계(억원)', '부채총계(억원)', '자본총계(억원)', '부채비율(%)']
        def _asset(_df_:DataFrame) -> DataFrame:
            _df_ = _df_[cols]
            _df_.columns = [col.replace("(억원)", "") for col in _df_]
            return _df_.dropna(how='all')

        a_asset = _asset(a)
        a_asset = a_asset[~a_asset.index.str.endswith("(E)")]
        q_asset = _asset(q)
        if q_asset.index[-1] == a_asset.index[-1]:
            asset = a_asset
        else:
            asset = concat([a_asset, q_asset.iloc[[-1]]], axis=0)
        asset = asset.iloc[-5:]
        obj = {
            "index": asset.index.tolist(),
            "asset": asset["자산총계"].tolist(),
            "assetText": [krw2currency(1e8 * v) for v in asset["자산총계"]],
            "capital": asset["자본총계"].tolist(),
            "capitalText": [krw2currency(1e8 * v) for v in asset["자본총계"]],
            "debt": asset["부채총계"].tolist(),
            "debtText": [krw2currency(1e8 * v) for v in asset["부채총계"]],
            "debtRatio": asset['부채비율(%)'].tolist()
        }
        return dumps(obj).replace("NaN", "null")

    @classmethod
    def convertGrowth(cls, a:DataFrame) -> str:
        yy = a.copy()
        yy = yy[yy.columns[:2].tolist() + ['EPS(원)']]
        yy = yy.dropna(how='all', axis=0)
        est = yy[yy.index.str.endswith('(E)')]
        yy = yy.drop(index=est.index)
        if not est.empty:
            yy = concat([yy, est.iloc[[0]]], axis=0)
        yy = round(100 * yy.pct_change().dropna(how='all', axis=0), 2)
        obj = {
            "date": yy.index.tolist(),
            "revenue": yy[yy.columns[0]].tolist(),
            "profit": yy[yy.columns[1]].tolist(),
            "eps": yy[yy.columns[2]].tolist(),
        }
        return dumps(obj).replace("NaN", "null")

    @classmethod
    def convertDeviation(cls, typical:Series, trend:DataFrame) -> str:
        df = concat([typical, trend], axis=1)
        objs = {}
        for col in df.columns[1:]:
            align = df[[df.columns[0], col]].dropna(how='all')
            if align.empty:
                objs[col] = align
                continue
            avg = (abs(align[df.columns[0]] - align[col]).sum() / len(align))
            objs[col] = (align[df.columns[0]] - align[col]) / avg
        dev = concat(objs=objs, axis=1)
        obj = {}
        for col in dev:
            data = dev[col].dropna()
            if data.empty:
                obj[col] = {'empty': 'true'}
            obj[col] = {
                'date': 'srcDate' if col == "10년" else f'srcTrend["{col}"].x',
                'data': round(data, 2).tolist(),
                'empty': 'false'
            }
        return dumps(obj).replace(" ", "").replace("NaN", "null") \
                         .replace('"srcDate"', 'srcDate') \
                         .replace('"srcTrend', 'srcTrend') \
                         .replace('].x"', '].x') \
                         .replace('\\"', '"')

    @classmethod
    def convertPerBand(cls, perBand:DataFrame, general:Series) -> str:
        if perBand.empty:
            return "null"

        perBand.index = perBand.index.strftime("%Y-%m-%d")
        perBand.loc[str(general["date"])] = [int(general["close"])] + [None] * (len(perBand.columns) - 1)
        perBand = perBand.sort_index()
        obj = {'x': perBand.index.tolist()}
        obj.update({col: perBand[col].dropna().tolist() for col in perBand})
        return dumps(obj).replace("NaN", "null")

    @classmethod
    def convertForeignRate(cls, foreignRate:DataFrame, general:Series) -> str:
        if foreignRate.empty:
            return "null"
        obj = {}
        for col in foreignRate:
            date, name = col
            if not date in obj:
                obj[date] = {}
            obj[date]['x'] = foreignRate[col].dropna().index.strftime("%Y-%m-%d").tolist()
            obj[date][name] = foreignRate[col].dropna().tolist()
            if not str(general["date"]) in obj[date]['x']:
                obj[date]['x'].append(str(general["date"]))
                if "종가" in name:
                    obj[date][name].append(int(general["close"]))
        return dumps(obj).replace("NaN", "null")

    @classmethod
    def convertPbr(cls, yy:DataFrame, general:Series) -> str:
        yy = yy.dropna(how='all', axis=0)[['PBR(배)', 'BPS(원)']]
        est = yy[yy.index.str.endswith('(E)')]
        yy = yy.drop(index=est.index)
        yy.loc[f'{general["date"][:4]}/현재'] = [general["close"]/general["recentBPS"], general["recentBPS"]]
        yy = yy.iloc[-4:]
        if not est.empty:
            yy.loc[est.index.values[0]] = [general["close"]/est.iloc[0]['BPS(원)'], est.iloc[0]['BPS(원)']]
        obj = {
            'x': yy.index.tolist(),
            'pbr': yy['PBR(배)'].tolist(),
            'bps': yy['BPS(원)'].tolist()
        }
        return dumps(obj).replace("NaN", "null")

    @classmethod
    def convertDiv(cls, yy:DataFrame, general:Series) -> str:
        yy = yy.dropna(how='all', axis=0)[['배당수익률(%)', 'DPS(원)']]
        est = yy[yy.index.str.endswith('(E)')]
        yy = yy.drop(index=est.index)
        if not est.empty:
            yy.loc[est.index.values[0]] = [round(100 * est.iloc[0]['DPS(원)']/general["close"], 2), est.iloc[0]['DPS(원)']]

        obj = {
            'x': yy.index.tolist(),
            'div': yy['배당수익률(%)'].tolist(),
            'dps': yy['DPS(원)'].tolist(),
            'meta': ('배당수익률: ' + yy['배당수익률(%)'].astype(str) + '%').tolist(),
        }
        if not est.empty:
            obj['meta'][-1] += '<br>* 최근 종가 대비 추정 DPS'
        return dumps(obj).replace("NaN", "null")

    @classmethod
    def convertPeg(cls, yy:DataFrame, general:Series, price:Series):
        yy = yy.dropna(how='all', axis=0)
        est = yy[yy.index.str.endswith('(E)')]
        yy = yy.drop(index=est.index)

        pe = yy["PER(배)"]
        epsR = 100 * yy["EPS(원)"].pct_change().shift(-1)
        recp = concat([pe, epsR], axis=1)
        if not est.empty:
            recp.iloc[-1, -1] = 100 * (est.iloc[0]['EPS(원)'] - yy.iloc[-1]['EPS(원)']) / yy.iloc[-1]['EPS(원)']
        recp["PEG"] = recp.apply(lambda r: r["PER(배)"]/r["EPS(원)"] if r["EPS(원)"] > 0 else None, axis=1)
        recp.loc[f'{general["date"][:4]}/현재'] = [general["trailingPE"], general["estimatedEpsGrowth"], general["PEG"]]
        recp = round(recp.iloc[-5:], 2)

        recp["적자"] = recp["PER(배)"].apply(lambda x: "적자" if isna(x) else "")
        recp["성장"] = recp["EPS(원)"].apply(lambda x: "미제공" if isna(x) else "역성장" if x <= 0 else "")

        text = []
        for n, r in recp.iterrows():
            if not isna(r["PEG"]):
                text.append(r["PEG"])
            else:
                if r["적자"] and r["성장"]:
                    text.append(f'{r["적자"]}/{r["성장"]}')
                else:
                    text.append(f'{r["적자"]}{r["성장"]}')

        recp['text'] = text
        recp = recp.join(price)
        obj = {
            'x': recp.index.tolist(),
            'peg': recp["PEG"].tolist(),
            'text': recp['text'].tolist(),
            'close': recp['close'].tolist()
        }
        return dumps(obj).replace("NaN", "null")


    @classmethod
    def convertProduct(cls, product: DataFrame) -> str:
        product = product.iloc[-1]
        product = product[product != 0]
        etc = product[product.index.str.contains("기타(계)")]
        if len(etc) == 1 and etc.sum() < 0:
            product.drop(labels=etc.index, inplace=True)
            product = product + (etc.sum() / len(product))
            product = round(product, 2)
            diff = round((100 - product.sum()), 2)
            if diff > 0:
                product.iloc[0] += diff
            else:
                product.iloc[0] -= diff
        if (len(etc) > 1) or (product.sum() != 100):
            return "null"
        obj = {'label': product.index.tolist(), 'value': product.values.tolist()}
        return dumps(obj).replace(" ", "").replace("NaN", "null")

    @classmethod
    def convertOverview(cls, baseline:Series) -> dict:
        obj = {
            'sector':  f'{baseline["sectorName"]}',
            'industry': f'{baseline["industryName"]}',
            'close': f'{baseline["close"]:,d}원',
            'marketCap': f'{krw2currency(baseline["marketCap"])}원',
            'volume': f'{baseline["volume"]:,d}주',
            'foreignRate': f'{baseline["foreignRate"]:.2f}%',
            'revenueType': f'{baseline["revenueType"]}',
            'revenue': f'{krw2currency(1e8 * baseline["trailingRevenue"])}원',
            'profit': f'{krw2currency(1e8 * baseline["trailingProfit"])}원',
            'profitRate': f'{baseline["trailingProfitRate"]:.2f}%',
            'estProfitRate': f'{baseline["estimatedProfitRate"]:.2f}%',
            'h52': f'{int(baseline["fiftyTwoWeekHigh"]):,d}원',
            'l52': f'{int(baseline["fiftyTwoWeekLow"]):,d}원',
            'return1Day': f'{"▼" if baseline["return1Day"] < 0 else "▲"}{baseline["return1Day"]:.2f}%',
            'color1Day': '#1861A8' if baseline["return1Day"] < 0 else '#C92A2A',
            'return1Week': f'{"▼" if baseline["return1Week"] < 0 else "▲"}{baseline["return1Week"]:.2f}%',
            'color1Week': '#1861A8' if baseline["return1Week"] < 0 else '#C92A2A',
            'return1Month': f'{"▼" if baseline["return1Month"] < 0 else "▲"}{baseline["return1Month"]:.2f}%',
            'color1Month': '#1861A8' if baseline["return1Month"] < 0 else '#C92A2A',
            'return3Month': f'{"▼" if baseline["return3Month"] < 0 else "▲"}{baseline["return3Month"]:.2f}%',
            'color3Month': '#1861A8' if baseline["return3Month"] < 0 else '#C92A2A',
            'return6Month': f'{"▼" if baseline["return6Month"] < 0 else "▲"}{baseline["return6Month"]:.2f}%',
            'color6Month': '#1861A8' if baseline["return6Month"] < 0 else '#C92A2A',
            'return1Year': f'{"▼" if baseline["return1Year"] < 0 else "▲"}{baseline["return1Year"]:.2f}%',
            'color1Year': '#1861A8' if baseline["return1Year"] < 0 else '#C92A2A',
            'eps': '미제공' if isna(baseline["trailingEps"]) else f'{int(baseline["trailingEps"]):,d}원',
            'estEps': "미제공",
            'bps': f'{int(baseline["recentBPS"]):,d}원',
            'per': f'{baseline["trailingPE"]:.2f}',
            'estPer': "미제공",
            'pbr': f'{baseline["priceToBook"]:.2f}',
        }
        if obj["close"] == obj["h52"]:
            obj['pct52h'] = "최고가"
            obj['pct52hColor'] = '#C92A2A'
        else:
            obj['pct52h'] = f'▼{baseline["pctFiftyTwoWeekHigh"]:.2f}%'
            obj['pct52hColor'] = '#1861A8'
        if obj["close"] == obj["l52"]:
            obj['pct52l'] = "최저가"
            obj['pct52lColor'] = '#1861A8'
        else:
            obj['pct52l'] = f'▲{baseline["pctFiftyTwoWeekLow"]:.2f}%'
            obj['pct52lColor'] = '#C92A2A'
        if not baseline["estimatedProfitRate"]:
            obj['estProfitRate'] = "미제공"
        if (not baseline["estimatedEps"] is None) and (not isna(baseline["estimatedEps"])):
            obj['estEps'] = f'{int(baseline["estimatedEps"]):,d}원'
            obj['estPer'] = f'{baseline["estimatedPE"]:.2f}'
        if isna(baseline['trailingRevenue']):
            obj['revenue'] = f'{krw2currency(1e8 * baseline["fiscalRevenue"])}원'
        if isna(baseline['trailingProfit']):
            obj['profit'] = f'{krw2currency(1e8 * baseline["fiscalProfit"])}원'
        if isna(baseline['trailingProfitRate']):
            obj['profitRate'] = f'{baseline["fiscalProfitRate"]:.2f}%'
        if isna(baseline['trailingPE']):
            obj['per'] =  f'{baseline["fiscalPE"]:.2f}'
        return obj

    def convertPer(self, baseline:Series) -> str:
        y = []
        text = []
        meta = []

        if isna(baseline["fiscalPE"]):
            y.append(0)
            text.append("적자" if baseline["fiscalEps"] <= 0 else "미제공")
        else:
            y.append(float(baseline["fiscalPE"]))
            text.append(f'{float(baseline["fiscalPE"])}')
        meta.append(f'직전회계연도({baseline["fiscalDate"]}) EPS 대비 종가')

        if isna(baseline["trailingPE"]):
            y.append(0)
            text.append("적자" if baseline["trailingEps"] <= 0 else "미제공")
        else:
            y.append(round(baseline["trailingPE"], 2))
            text.append(f'{round(baseline["trailingPE"], 2)}')
        meta.append(f'4분기연속EPS합산 대비 현재가(종가)')

        if isna(baseline["estimatedPE"]):
            y.append(0)
            if baseline["estimatedEps"] <= 0:
                text.append("적자")
                meta.append(f'추정회계연도({baseline["estimatedDate"].replace("(E)", "")}) EPS 대비 현재가(종가)')
            else:
                text.append("미제공")
                meta.append(f'증권사 추정치 미제공')
        else:
            y.append(round(baseline["estimatedPE"], 2))
            text.append(f'{round(baseline["estimatedPE"], 2)}')
            meta.append(f'추정회계연도({baseline["estimatedDate"].replace("(E)", "")}) EPS 대비 현재가(종가)')

        if isna(baseline["weightedAverageEps"]):
            y.append(0)
            text.append("적자" if baseline["weightedAverageEps"] <= 0 else "미제공")
        else:
            y.append(float(round(baseline["close"] / baseline["weightedAverageEps"], 2)))
            text.append(f'{float(round(baseline["close"] / baseline["weightedAverageEps"], 2))}')
        meta.append(f'{int(baseline["numberOfAnnualStatement"])}개 회계연도의 EPS 가중 평균 대비 현재가(종가)')

        y.append(round(self.basis[self.basis["industryCode"] == baseline["industryCode"]]["trailingPE"].mean(), 2))
        text.append(f'{y[-1]}')
        meta.append(f'{baseline["industryName"]} 업종 4분기연속 PE에 대한 평균')

        obj = {
            "x": ["직전회계연도PE", "4분기연속PE", "추정PE", "가중평균PE", "업종평균PE"],
            "y": y,
            "text": text,
            "meta": meta
        }
        return dumps(obj).replace("NaN", "null")


if __name__ == "__main__":
    from pandas import set_option
    set_option('display.expand_frame_repr', False)


    stocks = Stocks()
    for ticker, stock in stocks:
        print(stock.spec)



AttributeError: No such attribute: MARKET_CAP